In [ ]:
from datetime import datetime

import biogeme.biogeme as bio
import biogeme.database as db
import biogeme.segmentation as seg
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from biogeme.expressions import Beta, bioDraws, log, MonteCarlo, Variable, PanelLikelihoodTrajectory, exp, bioMultSum
from biogeme.models import loglogit, logit
from factor_analyzer import FactorAnalyzer
from scipy.stats import t



# Create dataset
Follow model of "airline.dat" from Biogeme.  
Column for Subject ID, each attribute with 1 or 2, then the choice


In [ ]:
# Read in the experimental design
Qualtrics_design_no_vac_COVID = pd.read_csv('/DCE_Analysis_NPIs/Qualtrics_design_no_vacc_COVID-19_29_April.csv')

# Pivot table so that each row is a question and each column is what was available in each alternative scenario 
Qualtrics_design_no_vac_COVID_pivoted = Qualtrics_design_no_vac_COVID.pivot_table(index=['question_set', 'question'],
                        columns='choice',
                        values=['Mask mandates', 'School closures', 'Business closures', 'Transit', 'Number infected', 'Healthcare restrictions'],
                        aggfunc='first')

Qualtrics_design_no_vac_COVID_pivoted.columns = ['_'.join(map(str, col)) for col in Qualtrics_design_no_vac_COVID_pivoted.columns.values]
Qualtrics_design_no_vac_COVID_pivoted.reset_index(inplace=True)
Qualtrics_design_no_vac_COVID_pivoted.rename(columns={'question_set': 'Question_Set'}, inplace=True)
Qualtrics_design_no_vac_COVID_pivoted.rename(columns={'question': 'Question'}, inplace=True)

# Add an empty "Chosen_Alternative" column
Qualtrics_design_no_vac_COVID_pivoted['Chosen_Alternative'] = None
Qualtrics_design_no_vac_COVID_pivoted.columns = Qualtrics_design_no_vac_COVID_pivoted.columns.str.replace(' ', '_')


 Read in participant responses

In [ ]:
study_data1 = pd.read_csv('DCE_Analysis_NPIs/DCE COVID 19 No Vaccine Abbreviated_June 6, 2024_09.39.csv')
study_data2 = pd.read_csv('DCE_Analysis_NPIs/DCE COVID 19 No Vaccine Abbreviated_January 13, 2025_07.47.csv')

study_data = pd.concat([study_data1, study_data2], ignore_index= True)
study_data = study_data.drop(index=len(study_data1) ).reset_index(drop=True)
study_data = study_data.drop(index=len(study_data1)).reset_index(drop=True) # no plus one as has been dropped
study_data = study_data.drop(index=0).reset_index(drop=True)

study_data.to_csv('DCE_Analysis_NPIs/DCE_Analysis_NPIs/study_data_no_vaccine.csv', index=False)


Find which version number they were using 

In [ ]:
version_number = study_data['vers_CBCONJOINT']
# remove first two rows
version_number = version_number[2:].reset_index(drop=True)

In [ ]:
# delete the first 18 columns - data on browers, etc. Only focussing on choice text for the moment - remove the other columns
study_data = study_data.iloc[:, 28:62]
# remove first two rows - give metadata on the question asked, etc
study_data = study_data.iloc[2:, :]
# reset the index
study_data.reset_index(drop=True, inplace=True)
# add the version number that each participant was asked to the dataframe
study_data['version_number'] = version_number
study_data['version_number'] = study_data['version_number'].astype(int)
study_data.rename(columns={'version_number': 'Question_Set'}, inplace=True)
#rename the supplementary questions
study_data = study_data.rename(columns = {"Q9": "Gender", "Q11": "Race", "Q12": "Hispanic", "Q13": "Age", "Q14": "Income", "Q15":"Residence", "Q16": "Child", "Q17": "Assisted_Living", "Q18":"State", "Q20":"Vaccine", "Q21":"Chronic", "Q34":"Vulnerable_contact", "Q22":"Health_Insurance", "Q24":"News", "Q25":"Political", "Q27":"Self_employed", "Q28":"Remote", "Q29":"Education", "Q30":"Pregnant", "Q31":"Vehicle" })
#remove text data 
study_data = study_data.drop(study_data.filter(regex='TEXT').columns, axis=1)
study_data = study_data.drop(study_data.filter(regex='Q10').columns, axis=1)


In [ ]:
participant_data = pd.DataFrame(columns=Qualtrics_design_no_vac_COVID_pivoted.columns[2:])
num_participants = study_data.shape[0]
num_questions = 10 #the number of DCE each participant does
# Assign the number of rows to participant_data
participant_data = participant_data.reindex(range(num_participants*num_questions))

In [ ]:
total_rows = num_participants * num_questions


In [ ]:
participant_data['Participant_ID'] = [i for i in range(1, num_participants + 1) for _ in range(num_questions)]
participant_data['Question_Number'] = [i % num_questions + 1 for i in range(total_rows)]


In [ ]:
for participant in range(0, len(study_data)):
     # match the version number to the correct experimental design
     version_number = study_data['Question_Set'][participant]
    # match the version number to the correct experimental design
     experimental_design = Qualtrics_design_no_vac_COVID_pivoted[Qualtrics_design_no_vac_COVID_pivoted['Question_Set'] == version_number]
    # reset the index
     experimental_design.reset_index(drop=True, inplace=True)
     index_for_input = 0
    # put the experimental design into the participant_data based on the participant number and question number 
     for question in range(0,len(experimental_design)):
         participant_data.loc[(participant*num_questions) + question, 'Question_Set'] = version_number
         participant_data.loc[(participant*num_questions) + question, 'Question'] = int(experimental_design['Question'][question])
         participant_data.loc[(participant*num_questions) + question, 'Mask_mandates_1'] = int(experimental_design['Mask_mandates_1'][question])
         participant_data.loc[(participant*num_questions) + question, 'Mask_mandates_2'] = int(experimental_design['Mask_mandates_2'][question])
         participant_data.loc[(participant*num_questions) + question, 'School_closures_1'] = int(experimental_design['School_closures_1'][question])
         participant_data.loc[(participant*num_questions) + question, 'School_closures_2'] = int(experimental_design['School_closures_2'][question])
         participant_data.loc[(participant*num_questions) + question, 'Business_closures_1'] = int(experimental_design['Business_closures_1'][question])
         participant_data.loc[(participant*num_questions) + question, 'Business_closures_2'] = int(experimental_design['Business_closures_2'][question])
         participant_data.loc[(participant*num_questions) + question, 'Transit_1'] = int(experimental_design['Transit_1'][question])
         participant_data.loc[(participant*num_questions) + question, 'Transit_2'] = int(experimental_design['Transit_2'][question])
         participant_data.loc[(participant*num_questions) + question, 'Number_infected_1'] = int(experimental_design['Number_infected_1'][question])
         participant_data.loc[(participant*num_questions) + question, 'Number_infected_2'] = int(experimental_design['Number_infected_2'][question])
         participant_data.loc[(participant*num_questions) + question, 'Healthcare_restrictions_1'] = int(experimental_design['Healthcare_restrictions_1'][question])
         participant_data.loc[(participant*num_questions) + question, 'Healthcare_restrictions_2'] = int(experimental_design['Healthcare_restrictions_2'][question])
         participant_data.loc[(participant*num_questions) + question, 'Chosen_Alternative'] = int(study_data.iloc[participant, question])

     
numeric_columns = ['Mask_mandates_1', 'Mask_mandates_2', 'School_closures_1', 'School_closures_2', 
                   'Business_closures_1', 'Business_closures_2', 'Transit_1', 
                   'Transit_2', 'Number_infected_1', 'Number_infected_2', 
                   'Healthcare_restrictions_1', 'Healthcare_restrictions_2', 'Chosen_Alternative']

participant_data[numeric_columns] = participant_data[numeric_columns].apply(pd.to_numeric)

         
 

Merge with socio economic factors

In [ ]:
study_data["Participant_ID"] = range(1, num_participants + 1)
participant_data = participant_data.merge(study_data.loc[:, ['Participant_ID', "Gender",  "Race", "Hispanic", "Age", "Income", "Residence", "Child", "Assisted_Living", "State","Vaccine", "Chronic","Vulnerable_contact", "Health_Insurance", "News", "Political", "Self_employed", "Remote", "Education", "Pregnant", "Vehicle"]], on="Participant_ID")


Recode the supplementary questions

In [ ]:
SE_columns = [ "Gender",  "Race", "Hispanic", "Age", "Income", "Residence", "Child", "Assisted_Living", "State","Vaccine", "Chronic","Vulnerable_contact","Health_Insurance", "News", "Political", "Self_employed", "Remote", "Education", "Pregnant", "Vehicle"]
SE_columns_binary = [ "Hispanic","Child", "Assisted_Living","Vaccine","Vulnerable_contact", "Health_Insurance", "Self_employed", "Education", "Pregnant", "Vehicle"]

SE_columns_multi = [ "Gender",  "Race", "Age", "Income", "Residence", "State","News", "Chronic","Political", "Remote", "Education"]
participant_data[SE_columns_binary] = participant_data[SE_columns_binary].replace('Yes', 1)
participant_data[SE_columns_binary] = participant_data[SE_columns_binary].replace('No', 2)

gender_mapping = {'Female': 1, 'Male': 2, 'Non-binary': 3, 'Self-define':4, 'Prefer not to answer': 5}
race_mapping = {
    'Black or African American': 1, 'Native Hawaiian or Pacific Islander': 2, 'American Indian or Native American or Alaskan Native': 3, 
    'Asian/Asian American': 4, 'Middle Eastern of North African': 5, 'Chinese': 6, 'Vietnamese': 7, 
    'Filipino': 8, 'Samoan': 9, 'Asian Indian': 10, 'Japanese': 11, 'Chamoan': 12, 'White': 13, 'Other': 14, 'Self-defined': 15
}
chronic_mapping = {'Yes': 1, 'No': 2, 'Prefer not to answer': 3}
age_mapping = {'18-24': 1, '25-34': 2, '35-49': 3, '50-64': 4, '≥ 65': 5}
income_mapping = {'< $35,000': 1, '$35,000 - 75,000': 2, '$75,000 - 150,000': 3, '> $150,000': 4}
residence_mapping = {'Urban': 1, 'Suburban': 2, 'Rural': 3}
#state_mapping = {state: i + 1 for i, state in enumerate(['Alabama', 'Arkansas', 'Arizona', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Iowa', 'Idaho', 'Illinois', 'Indiana', 'Kansas', 'Kentucky', 'Louisiana', 'Massachusetts', 'Maryland', 'Maine', 'Michigan', 'Minnesota', 'Missouri', 'Mississippi', 'Montana', 'North Carolina', 'North Dakota', 'Nebraska', 'New Hampshire', 'New Jersey', 'New Mexico', 'Nevada', 'New York', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Virginia', 'Vermont', 'Washington', 'Wisconsin', 'West Virginia', 'Wyoming', 'Alaska', "Hawai'i", 'District of Columbia'])}
state_mapping = {
    'Missouri': 1,
    'Mississippi': 2,
    'Ohio': 3,
    'Illinois': 4,
    'South Carolina': 5,
    'Tennessee': 6,
    'New York': 7,
    'Texas': 8,
    'Pennsylvania': 9,
    'Arizona': 10,
    'Kentucky': 11,
    'Alabama': 12,
    'Georgia': 13,
    'Michigan': 14,
    'California': 15,
    'Florida': 16,
    'Virginia': 17,
    'Arkansas': 18,
    'Washington': 19,
    'Utah': 20,
    'Oklahoma': 21,
    'Nebraska': 22,
    'Minnesota': 23,
    'North Caroline': 24,
    'Nevada': 25,
    'Oregon': 26,
    'Louisiana': 27,
    'South Dakota': 28,
    'Colorado': 29,
    'Connecticut': 30,
    'New Mexico': 31,
    'West Virginia': 32,
    'Indiana': 33,
    'Maryland': 34,
    'Alaska': 35,
    'Iowa': 36,
    'Massachusetts': 37,
    'Kansas': 38,
    'Idaho': 39,
    'Wisconsin': 40,
    'New Jersey': 41,
    'Washington DC': 42,
    'Delaware': 43,
    "Hawai'i": 44,
    'Montana': 45,
    'New Hampshire': 46,
    'North Dakota': 47,
    'Rhode Island': 48,
    'Maine': 49, 
    'Wyoming': 50
}
news_mapping = {'Social media (Instagram, Facebook, X (Twitter), TikTok)': 1, 'Print media (newspapers, journals)': 2, 'TV (including cable)': 3, 'Radio or podcasts': 4, 'News apps or websites': 5, 'Do not read/listen/watch the news':6, 'Other': 7}
political_mapping = {'Democrat': 1, 'Republican': 2, 'Independent': 3, 'Other': 4}
education_mapping = {
    'High School or GED': 1, 'Some college credits': 2, '1 or more years of college credit, no degree': 3, 
    'Associate’s degree': 4, 'Undergraduate degree': 5, 'Postgraduate degree': 6
}
remote_mapping = {'Yes': 1, 'No': 2, 'NA (studying, retired, not in paid employment)':3}

# Apply custom mappings
participant_data['Gender'] = participant_data['Gender'].map(gender_mapping)
participant_data['Race'] = participant_data['Race'].map(race_mapping)
participant_data['Age'] = participant_data['Age'].map(age_mapping)
participant_data['Income'] = participant_data['Income'].map(income_mapping)
participant_data['Residence'] = participant_data['Residence'].map(residence_mapping)
participant_data['State'] = participant_data['State'].map(state_mapping)
participant_data['News'] = participant_data['News'].map(news_mapping)
participant_data['Political'] = participant_data['Political'].map(political_mapping)
participant_data['Education'] = participant_data['Education'].map(education_mapping)
participant_data['Remote'] = participant_data['Remote'].map(remote_mapping)
participant_data['Chronic'] = participant_data['Chronic'].map(chronic_mapping)

participant_data['Race'].fillna(16, inplace = True)
participant_data['Pregnant'].fillna(2, inplace = True)

Check 

In [ ]:
for column in participant_data.columns:
    nan_rows = participant_data[participant_data[column].isna()]
    if not nan_rows.empty:
        print(f"NaN values in column '{column}':")

# fill na with 0 

participant_data.fillna(0, inplace=True)



Create a database object


In [ ]:
participant_data_db = db.Database("participant_data_db", participant_data)

# Create the variables, utilities, alternative-specific constants, utilities, coefficients

Create variables

In [ ]:
globals().update(participant_data_db.variables)


In [ ]:
mask_mandates_1 = Variable('Mask_mandates_1')
mask_mandates_2 = Variable('Mask_mandates_2')
school_closures_1 = Variable('School_closures_1')
school_closures_2 = Variable('School_closures_2')
business_closures_1 = Variable('Business_closures_1')
business_closures_2 = Variable('Business_closures_2')
Transit_1 = Variable('Transit_1')
Transit_2 = Variable('Transit_2')
Number_infected_1 = Variable('Number_infected_1')
Number_infected_2 = Variable('Number_infected_2')
healthcare_restrictions_1 = Variable('Healthcare_restrictions_1')
healthcare_restrictions_2 = Variable('Healthcare_restrictions_2')
chosen_alternative = Variable('Chosen_Alternative')

Parameters for estimation 

In [ ]:
# Parameters to be estimated
# Arguments:
#   1  Name for report. Typically, the same as the variable
#   2  Starting value
#   3  Lower bound
#   4  Upper bound
#   5  0: estimate the parameter, 1: keep it fixed


# ASC for "Alternative 2"
ACS2 = Beta('ACS2', 0, None, None, 0)

# Attributes
mask_mandates = Beta('Mask_mandates', 0, None, None, 0)
school_closures = Beta('School_closures', 0, None, None, 0)
business_closures = Beta('Business_closures', 0, None, None, 0)
transit_2 = Beta('transit_2', 0, None, None, 0)
number_of_infections = Beta('Number_of_infections', 0, None, None, 0)
healthcare_restrictions = Beta('Healthcare_restrictions', 0, None, None, 0)



Utilities

In [ ]:
# Utilities
Alt1 = (
    mask_mandates * mask_mandates_1
    + school_closures * school_closures_1
    + business_closures * business_closures_1
    + transit_2 * Transit_1
    + number_of_infections * Number_infected_1
    + healthcare_restrictions * healthcare_restrictions_1
)
Alt2 = (ACS2 + 
    mask_mandates * mask_mandates_2
    + school_closures * school_closures_2
    + business_closures * business_closures_2
    + transit_2 * Transit_2
    + number_of_infections * Number_infected_2
    + healthcare_restrictions * healthcare_restrictions_2
)

# Associate utility functions with the numbering of alternatives
V = {1: Alt1, 2: Alt2}
# Associate the availability conditions with the alternatives

av = {1: 1, 2: 1}



Define the multinomial logit model 

In [ ]:

# Definition of the model. This is the contribution of each
# observation to the log likelihood function.
logprob = loglogit(V, av, chosen_alternative)

# Create the Biogeme object
biogeme = bio.BIOGEME(participant_data_db, logprob)
biogeme.modelName = '01logit'

# Estimate the parameters
results_no_one_hot_encode = biogeme.estimate()

# Get the results in a pandas table
pandasResults_no_one_hot_encode = results_no_one_hot_encode.getEstimatedParameters()
print(pandasResults_no_one_hot_encode)
print(f"Nbr of observations: {participant_data_db.getNumberOfObservations()}")
print(f"LL(0) =    {results_no_one_hot_encode.data.initLogLike:.3f}")
print(f"LL(beta) = {results_no_one_hot_encode.data.logLike:.3f}")
print(f"rho bar square = {results_no_one_hot_encode.data.rhoBarSquare:.3g}")
print(f"Output file: {results_no_one_hot_encode.data.htmlFileName}")

# One-hot encode all categorical variables (all of them except infection risk)

In [ ]:
columns_to_encode = ['Mask_mandates_1', 'Mask_mandates_2', 'School_closures_1', 'School_closures_2',
                     'Business_closures_1', 'Business_closures_2', 'Transit_1',
                     'Transit_2',
                     'Healthcare_restrictions_1', 'Healthcare_restrictions_2']


# One-hot encode the specified columns
encoded_data = pd.get_dummies(participant_data, columns=columns_to_encode, drop_first=False) #True)
encoded_data = encoded_data.astype(int)
encoded_data_db = db.Database("encoded_data_db", encoded_data)

Change the labels of the infection data to make it "continuous"

In [ ]:
for row in range(0, len(encoded_data)):
    if encoded_data.loc[row, 'Number_infected_1'] == 5:
        encoded_data.loc[row, 'Number_infected_1'] == 0.20
    elif encoded_data.loc[row, 'Number_infected_1'] == 4:
        encoded_data.loc[row, 'Number_infected_1'] == 0.15
    elif encoded_data.loc[row, 'Number_infected_1'] == 3:
        encoded_data.loc[row, 'Number_infected_1'] == 0.10
    elif encoded_data.loc[row, 'Number_infected_1'] == 2:
        encoded_data.loc[row, 'Number_infected_1'] == 0.5
    elif encoded_data.loc[row, 'Number_infected_1'] == 1:
        encoded_data.loc[row, 'Number_infected_1'] == 0.1

# Define variables, constants, parameters

In [ ]:
globals().update(encoded_data_db.variables)


set as panel data 

In [ ]:
encoded_data_db.panel('Participant_ID')

Only ASC2 and random parameter for panel data has biodraws

Create coefficients

In [ ]:

mask_mandates_1 = Beta('Mask_mandates_1', 0, None, None, 0)
mask_mandates_2 = Beta('Mask_mandates_2', 0, None, None, 0)
mask_mandates_3 = Beta('Mask_mandates_3', 0, None, None, 0)
mask_mandates_4 = Beta('Mask_mandates_4', 0, None, None, 0)

school_closures_1 = Beta('School_closures_1', 0, None, None, 0)
school_closures_2 = Beta('School_closures_2', 0, None, None, 0)
school_closures_3 = Beta('School_closures_3', 0, None, None, 0)
school_closures_4 = Beta('School_closures_4', 0, None, None, 0)
school_closures_5 = Beta('School_closures_5', 0, None, None, 0)

business_closures_1 = Beta('Business_closures_1', 0, None, None, 0)
business_closures_2 = Beta('Business_closures_2', 0, None, None, 0)
business_closures_3 = Beta('Business_closures_3', 0, None, None, 0)

Transit_1 = Beta('Transit_1', 0, None, None, 0)
Transit_2 = Beta('Transit_2', 0, None, None, 0)

number_of_infections = Beta('Number_of_infections', 0, None, None, 0)

healthcare_restrictions_1 = Beta('Healthcare_restrictions_1', 0, None, None, 0)
healthcare_restrictions_2 = Beta('Healthcare_restrictions_2', 0, None, None, 0)
healthcare_restrictions_3 = Beta('Healthcare_restrictions_3', 0, None, None, 0)


Below matches the specification in https://github.com/pmontman/tmp_choicemodels/blob/b5c800d5427b35bcf1ac79ae2e39926d9dbaab6d/nb/tutorials/solutions/WK_07_solu_tuto_panel_mixed_logit.ipynb

In [ ]:


# ASC for "Alternatives"
ACS1 = Beta('ACS1', 0, None, None, 0)
ACS2 = Beta('ACS2', 0, None, None, 0)

SIGMA_PANEL_ASC1 = Beta('SIGMA_PANEL_ASC1', 1, None, None, 0)

#ACS1_param = ACS1 + SIGMA_PANEL_ASC1 * bioDraws(
#    'ACS2_param', 'NORMAL'
#)

SIGMA_PANEL_1 = Beta('SIGMA_PANEL_1', 1, None, None, 0)

# Random parameters for Panel data
ZERO_SIGMA_PANEL_1 = SIGMA_PANEL_1 * bioDraws(
    'ZERO_SIGMA_PANEL_1', 'NORMAL'
)

#ACS2_param = ACS2 + SIGMA_PANEL_ASC2 * bioDraws(
#    'ACS2_param', 'NORMAL'
#)
SIGMA_PANEL_2 = Beta('SIGMA_PANEL_2', 1, None, None, 0)

# Random parameters for Panel data
ZERO_SIGMA_PANEL_2 = SIGMA_PANEL_2 * bioDraws(
    'ZERO_SIGMA_PANEL_2', 'NORMAL'
)
# Utilities
Alt1 = (ACS1 + ZERO_SIGMA_PANEL_1 + 
     #mask_mandates_1 * Mask_mandates_1_1
    mask_mandates_2 * Mask_mandates_1_2
    + mask_mandates_3 * Mask_mandates_1_3
    + mask_mandates_4 * Mask_mandates_1_4

    #+ school_closures_1* School_closures_1_1
    + school_closures_2* School_closures_1_2
    + school_closures_3  * School_closures_1_3
    + school_closures_4  * School_closures_1_4
    + school_closures_5  * School_closures_1_5

    #+ business_closures_1  * Business_closures_1_1
    + business_closures_2  * Business_closures_1_2
    + business_closures_3  * Business_closures_1_3

    #+ Transit_1 * Transit_1_1
    + Transit_2 * Transit_1_2

    + number_of_infections  * Number_infected_1

    #+ healthcare_restrictions_1  * Healthcare_restrictions_1_1
    + healthcare_restrictions_2  * Healthcare_restrictions_1_2
    + healthcare_restrictions_3  * Healthcare_restrictions_1_3
)
Alt2 = (ACS2 + ZERO_SIGMA_PANEL_2 + 
    #mask_mandates_1 * Mask_mandates_2_1
    mask_mandates_2  * Mask_mandates_2_2
    + mask_mandates_3  * Mask_mandates_2_3
    + mask_mandates_4  * Mask_mandates_2_4

    #+ school_closures_1    * School_closures_2_1
    + school_closures_2  * School_closures_2_2
    + school_closures_3  * School_closures_2_3
    + school_closures_4  * School_closures_2_4
    + school_closures_5  * School_closures_2_5

    
    #+ business_closures_1 * Business_closures_2_1
    + business_closures_2 * Business_closures_2_2
    + business_closures_3  * Business_closures_2_3
    
    #+ Transit_1 * Transit_2_1
    + Transit_2  * Transit_2_2

    + number_of_infections  * Number_infected_2

    #+ healthcare_restrictions_1 * Healthcare_restrictions_2_1
    + healthcare_restrictions_2  * Healthcare_restrictions_2_2
    + healthcare_restrictions_3 * Healthcare_restrictions_2_3

)

# Associate utility functions with the numbering of alternatives
V = {1: Alt1, 2: Alt2}
# Associate the availability conditions with the alternatives

av = {1: 1, 2: 1}


In [ ]:
# Definition of the model. This is the contribution of each
# observation to the
obsprob = logit(V, av, chosen_alternative)
condprobIndiv = PanelLikelihoodTrajectory(obsprob)
logprob = log(MonteCarlo(condprobIndiv))
# Create the Biogeme object
biogeme = bio.BIOGEME(encoded_data_db, logprob, numberOfDraws=500)
biogeme.modelName = '01logit_no_bio'

# Estimate the parameters
biogeme.calculateNullLoglikelihood(av)
start_time = datetime.now()
results = biogeme.estimate()
print(f'Estimation time: {datetime.now() - start_time}')

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)
print(f'Nbr of observations: {encoded_data_db.getNumberOfObservations()}')
print(f'LL(0) =    {results.data.initLogLike:.3f}')
print(f'LL(beta) = {results.data.logLike:.3f}')
print(f'AIC = {results.data.akaike:.3f}')
print(f'Output file: {results.data.htmlFileName}')

# Flatten database for panel data

In [ ]:
flat_df = encoded_data_db.generateFlatPanelDataframe(identical_columns=None)
flat_database = db.Database('encoded_data_db_flat', flat_df)

In [ ]:
globals().update(flat_database.variables)

In [ ]:
# ASC for "Alternative 2"
ACS2 = Beta('ACS2', 0, None, None, 0)
SIGMA_PANEL_ASC2 = Beta('SIGMA_PANEL_ASC2', 1, None, None, 0)

ACS2_param = ACS2 + SIGMA_PANEL_ASC2 * bioDraws(
    'ACS2_param', 'NORMAL'
)

ACS1 = Beta('ACS1', 0, None, None, 0)
SIGMA_PANEL_ASC1 = Beta('SIGMA_PANEL_ASC1', 1, None, None, 0)

ACS1_param = ACS1 + SIGMA_PANEL_ASC1 * bioDraws(
    'ACS1_param', 'NORMAL'
)

In [ ]:
# Utilities
Alt1 = [
    #mask_mandates_1 * Variable(f'{t}_Mask_mandates_1_1')
     mask_mandates_2 * Variable(f'{t}_Mask_mandates_1_2')
    + mask_mandates_3 * Variable(f'{t}_Mask_mandates_1_3')
    + mask_mandates_4 * Variable(f'{t}_Mask_mandates_1_4')

    
    #+ school_closures_1 * Variable(f'{t}_School_closures_1_1')
    + school_closures_2 * Variable(f'{t}_School_closures_1_2')
    + school_closures_3 * Variable(f'{t}_School_closures_1_3')
    + school_closures_4 * Variable(f'{t}_School_closures_1_4')
    + school_closures_5 * Variable(f'{t}_School_closures_1_5')

    #+ business_closures_1 * Variable(f'{t}_Business_closures_1_1')
    + business_closures_2 * Variable(f'{t}_Business_closures_1_2')
    + business_closures_3 * Variable(f'{t}_Business_closures_1_3')

    #+ Transit_1 * Variable(f'{t}_Transit_1_1')
    + Transit_2 * Variable(f'{t}_Transit_1_2')

    + number_of_infections * Variable(f'{t}_Number_infected_1')

    #+ healthcare_restrictions_1 * Variable(f'{t}_Healthcare_restrictions_1_1') 
    + healthcare_restrictions_2 * Variable(f'{t}_Healthcare_restrictions_1_2') 
    + healthcare_restrictions_3 * Variable(f'{t}_Healthcare_restrictions_1_3') 
     for t in range(1, 10)
]
Alt2 = [ACS2_param +  
    #mask_mandates_1 * Variable(f'{t}_Mask_mandates_2_1')
     mask_mandates_2 * Variable(f'{t}_Mask_mandates_2_2')
    + mask_mandates_3 * Variable(f'{t}_Mask_mandates_2_3')
    + mask_mandates_4 * Variable(f'{t}_Mask_mandates_2_4')

    #+ school_closures_1 * Variable(f'{t}_School_closures_2_1')
    + school_closures_2 * Variable(f'{t}_School_closures_2_2')
    + school_closures_3 * Variable(f'{t}_School_closures_2_3')
    + school_closures_4 * Variable(f'{t}_School_closures_2_4')
    + school_closures_5 * Variable(f'{t}_School_closures_2_5')

    #+ business_closures_1 * Variable(f'{t}_Business_closures_2_1')
    + business_closures_2 * Variable(f'{t}_Business_closures_2_2')
    + business_closures_3 * Variable(f'{t}_Business_closures_2_3')

    #+ Transit_1 * Variable(f'{t}_Transit_2_1')
    + Transit_2 * Variable(f'{t}_Transit_2_2')

    + number_of_infections * Variable(f'{t}_Number_infected_2')

    #+ healthcare_restrictions_1 * Variable(f'{t}_Healthcare_restrictions_2_1') 
    + healthcare_restrictions_2 * Variable(f'{t}_Healthcare_restrictions_2_2') 
    + healthcare_restrictions_3 * Variable(f'{t}_Healthcare_restrictions_2_3') 
    for t in range(1, 10)
]

# Associate utility functions with the numbering of alternatives
V = [{1: Alt1[t], 2: Alt2[t]} for t in range(9)]

# Associate the availability conditions with the alternatives

av = {1: 1, 2: 1}

obsprob = [loglogit(V[t], av, Variable(f'{t+1}_Chosen_Alternative')) for t in range(9)]
condprobIndiv = exp(bioMultSum(obsprob))

logprob = log(MonteCarlo(condprobIndiv))
the_biogeme = bio.BIOGEME(flat_database, logprob, parameter_file='few_draws.toml', numberOfDraws=500)
the_biogeme.modelName = 'b12panel_flat_N_total'

In [ ]:
results_no_drop_first_level= the_biogeme.estimate()


In [ ]:
pandas_results_drop_first_level = results_no_drop_first_level.getEstimatedParameters(onlyRobust=False)
pandas_results_drop_first_level['CI_lower'] = pandas_results_drop_first_level['Value'] - pandas_results_drop_first_level['Std err'] * 1.96
pandas_results_drop_first_level['CI_upper'] = pandas_results_drop_first_level['Value'] + pandas_results_drop_first_level['Std err'] * 1.96

print(pandas_results_drop_first_level)

Test parameters

In [ ]:
pandas_results_drop_first_level

In [ ]:

# Extract coefficients, standard errors, and covariance matrix
coefficients = pandasResults['Value']
std_errors = pandasResults['Rob. Std err']

# Initialize storage for pairwise t-tests
pairwise_p_values = []

# Perform pairwise t-tests
for i in range(len(coefficients)):
    for j in range(i + 1, len(coefficients)):
        # Calculate t-statistic for each pair
        t_stat = (coefficients[i] - coefficients[j]) / np.sqrt(std_errors[i]**2 + std_errors[j]**2)
        
        # Degrees of freedom approximation (assuming large sample size)
        df = len(coefficients) - 2
        
        # Calculate p-value
        p_value = 2 * (1 - t.cdf(np.abs(t_stat), df))
        
        # Store the results
        pairwise_p_values.append({
            'Coefficient1': i,
            'Coefficient2': j,
            'tStatistic': t_stat,
            'pValue': p_value
        })

# Convert to DataFrame
pairwise_p_values_df = pd.DataFrame(pairwise_p_values)

# Apply Bonferroni adjustment
num_tests = len(pairwise_p_values_df)
bonferroni_threshold = 0.05 / num_tests
pairwise_p_values_df['Bonferroni pValue'] = pairwise_p_values_df['pValue'] * num_tests
pairwise_p_values_df['Bonferroni pValue'] = pairwise_p_values_df['Bonferroni pValue'].clip(upper=1)
pairwise_p_values_df['Bonferroni Significant'] = pairwise_p_values_df['Bonferroni pValue'] < bonferroni_threshold


# Summary statistics 

In [ ]:
for column in SE_columns:
    # Count the frequency of each value in the column
    value_counts = study_data[column].value_counts()
    
    # Create a bar chart
    plt.figure(figsize=(10, 6))  # Adjust the figure size as needed
    ax = value_counts.plot(kind='bar', color='skyblue')
    plt.title(f'Bar Chart for {column}')
    plt.xlabel(column)
    plt.ylabel('Frequency')
    plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
    
    # Annotate each bar with its value and the value divided by 44 rounded to 2 decimal places
    for i, v in enumerate(value_counts):
        ax.text(i, v + 0.1, str(v), ha='center')
        ax.text(i, v - 100, str(round(float(v) / num_participants, 4)), ha='center')

    plt.tight_layout()
    plt.show()


In [ ]:
table_data = []

for column in SE_columns:
    # Count the frequency of each value in the column
    value_counts = study_data[column].value_counts()

    # Collect the data for the table
    for value, count in value_counts.items():
        proportion = round(float(count) / num_participants, 4)
        table_data.append([column, value, count, proportion])
        print([column, value, count, proportion])

results_table = pd.DataFrame(table_data, columns=['Column', 'Value', 'Frequency', 'Proportion'])

# Check sample size 

from https://www.erim.eur.nl/fileadmin/user_upload/R_code_generic_withcitation_20190517.txt

In [ ]:
covariance_matrix = results_no_drop_first_level.getRobustVarCovar()

rows_to_remove = [index for index in covariance_matrix.index if '_1' in index]
columns_to_remove = [column for column in covariance_matrix.columns if '_1' in column]


# Drop the rows and columns containing '_1'
#covariance_matrix_filtered = covariance_matrix.drop(index=rows_to_remove, columns=columns_to_remove)
rows_to_remove = [index for index in covariance_matrix.index if 'SIGMA_PANEL_ASC2' in index]
columns_to_remove = [column for column in covariance_matrix.columns if 'SIGMA_PANEL_ASC2' in column]
covariance_matrix_filtered = covariance_matrix.drop(index=rows_to_remove, columns=columns_to_remove)



covariance_matrix_filtered.to_csv('covariance_matrix_do_drop_first_level_no_vaccine.csv', index=False)

# Segmented based on socio-economic factors 

In [ ]:
gender_segmentation = seg.DiscreteSegmentationTuple(
    variable=Gender, mapping={1: 'Female', 2: 'Male', 3: 'Non-binary or Self-define or Prefer not to answer', 4: 'Non-binary or Self-define or Prefer not to answer', 5: 'Non-binary or Self-define or Prefer not to answer'}
)

Hispanic_segmentation = seg.DiscreteSegmentationTuple(
    variable=Hispanic, mapping={1: 'Yes', 2: 'No'}
)

age_segmentation = seg.DiscreteSegmentationTuple(
    variable=Age, mapping={1: '18-24', 1: '25-34', 1: '35-49', 1: '50-64', 2: '≥ 65'}
)

income_segmentation = seg.DiscreteSegmentationTuple(
    variable=Income, mapping={1: '< $35,000', 2: '$35,000 - 75,000', 3: '$75,000 - $150,000', 3: '> $150,000'}
)


residence_segmentation = seg.DiscreteSegmentationTuple(
    variable=Residence, mapping={1: 'Urban', 2: 'Suburban', 3: 'Rural'}
)

education_segmentation = seg.DiscreteSegmentationTuple(
    variable=Education, mapping={
        1: 'High School or GED', 2: 'Some college credits', 3: '1 or more years of college credit, no degree', 
        4: 'Associate’s degree', 5: 'Undergraduate degree', 6: 'Postgraduate degree'
    }
)
education_segmentation = seg.DiscreteSegmentationTuple(
    variable=Education, mapping={
        1: 'High School or GED', 1: 'Some college credits', 1: '1 or more years of college credit, no degree', 
        1: 'Associate’s degree', 2: 'Undergraduate degree', 2: 'Postgraduate degree'
    }
)
child_segmentation = seg.DiscreteSegmentationTuple(
    variable=Child, mapping={1: 'Yes', 2: 'No'}
)

vulnerable_contact_segmentation = seg.DiscreteSegmentationTuple(
    variable=Vulnerable_contact, mapping={1: 'Yes', 2: 'No'}
)
assisted_living_segmentation = seg.DiscreteSegmentationTuple(
    variable=Assisted_Living, mapping={1: 'Yes', 2: 'No'}
)

vaccine_segmentation = seg.DiscreteSegmentationTuple(
    variable=Vaccine, mapping={1: 'Yes', 2: 'No'}
)

chronic_segmentation = seg.DiscreteSegmentationTuple(
    variable=Chronic, mapping={1: 'Yes', 2: 'No', 3: 'Prefer not to answer'}
)

health_insurance_segmentation = seg.DiscreteSegmentationTuple(
    variable=Health_Insurance, mapping={1: 'Yes', 2: 'No'}
)

self_employed_segmentation = seg.DiscreteSegmentationTuple(
    variable=Self_employed, mapping={1: 'Yes', 2: 'No'}
)

remote_segmentation = seg.DiscreteSegmentationTuple(
    variable=Remote, mapping={1: 'Yes', 2: 'No', 3: 'NA (studying, retired, not in paid employment)'}
)
remote_segmentation = seg.DiscreteSegmentationTuple(
    variable=Remote, mapping={1: 'Yes', 2: 'No', 3: 'No'}
)

pregnant_segmentation = seg.DiscreteSegmentationTuple(
    variable=Pregnant, mapping={1: 'Yes', 2: 'No'}
)

vehicle_segmentation = seg.DiscreteSegmentationTuple(
    variable=Vehicle, mapping={1: 'Yes', 2: 'No'}
)

political_segmentation = seg.DiscreteSegmentationTuple(
    variable=Political, mapping={1: 'Democrat', 2: 'Republican', 3: 'Independent', 4: 'Other'}
)

news_segmentation = seg.DiscreteSegmentationTuple(
    variable=News, mapping={
        1: 'Social media (Instagram, Facebook, X (Twitter), TikTok)', 2: 'Print media (newspapers, journals)', 
        3: 'TV (including cable)', 4: 'Radio or podcasts', 5: 'News apps or websites', 6: 'Do not read/listen/watch the news', 7: 'Other'
    }
)
news_segmentation = seg.DiscreteSegmentationTuple(
    variable=News, mapping={
        1: 'Social media (Instagram, Facebook, X (Twitter), TikTok)', 2: 'Other', 
        3: 'TV (including cable)', 4: 'Radio or podcasts', 5: 'News apps or websites', 6: 'Other', 7: 'Other'
    }
)

state_segmentation = seg.DiscreteSegmentationTuple(
    variable=State, mapping={
        1: 'Missouri', 2: 'Mississippi', 3: 'Ohio', 4: 'Illinois', 5: 'South Carolina', 6: 'Tennessee', 7: 'New York',
        8: 'Texas', 9: 'Pennsylvania', 10: 'Arizona', 11: 'Kentucky', 12: 'Alabama', 13: 'Georgia', 14: 'Michigan',
        15: 'California', 16: 'Florida', 17: 'Virginia', 18: 'Arkansas', 19: 'Washington', 20: 'Utah', 21: 'Oklahoma',
        22: 'Nebraska', 23: 'Minnesota', 24: 'North Carolina', 25: 'Nevada', 26: 'Oregon', 27: 'Louisiana', 28: 'South Dakota',
        29: 'Colorado', 30: 'Connecticut', 31: 'New Mexico', 32: 'West Virginia', 33: 'Indiana', 34: 'Maryland', 35: 'Alaska',
        36: 'Iowa', 37: 'Massachusetts', 38: 'Kansas', 39: 'Idaho', 40: 'Wisconsin', 41: 'New Jersey', 42: 'Washington DC',
        43: 'Delaware', 44: "Hawai'i", 45: 'Montana', 46: 'New Hampshire', 47: 'North Dakota', 48: 'Rhode Island', 49: 'Maine', 50: 'Wyoming'
    }
)

race_segmentation = seg.DiscreteSegmentationTuple(
    variable=Race, mapping={
        1: 'Black or African American', 2: 'Native Hawaiian or Pacific Islander or American Indian or Native American or Alaskan Native or Chamoan or Samoan', 3: 'Native Hawaiian or Pacific Islander or American Indian or Native American or Alaskan Native or Chamoan or Samoan',
        4: 'Asian/Asian American or Chinese or Vietnamese or Filipino or Japanese or Asian Indian', 5: 'Middle Eastern of North African', 6: 'Asian/Asian American or Chinese or Vietnamese or Filipino or Japanese or Asian Indian', 7: 'Asian/Asian American or Chinese or Vietnamese or Filipino or Japanese or Asian Indian', 8: 'Asian/Asian American or Chinese or Vietnamese or Filipino or Japanese or Asian Indian',
        9: 'Native Hawaiian or Pacific Islander or American Indian or Native American or Alaskan Native or Chamoan or Samoan', 10: 'Asian/Asian American or Chinese or Vietnamese or Filipino or Japanese or Asian Indian', 11: 'Asian/Asian American or Chinese or Vietnamese or Filipino or Japanese or Asian Indian', 12: 'Native Hawaiian or Pacific Islander or American Indian or Native American or Alaskan Native or Chamoan or Samoan', 13: 'White', 14: 'Other or Self-defined or Multi-racial', 15: 'Other or Self-defined or Multi-racial', 16: 'Other or Self-defined or Multi-racial'
    }
)



Segment coefficients

In [ ]:
# gender segmentation
segmented_mask_mandates_2_gender = seg.Segmentation(
    mask_mandates_2,
    [gender_segmentation]
).segmented_beta()

segmented_mask_mandates_3_gender  = seg.Segmentation( 
    mask_mandates_3,
    [gender_segmentation]
).segmented_beta()

segmented_mask_mandates_4_gender  = seg.Segmentation(
    mask_mandates_4,
    [gender_segmentation]
).segmented_beta()

segmented_school_closures_2_gender  = seg.Segmentation(
    school_closures_2,
    [gender_segmentation]
).segmented_beta()

segmented_school_closures_3_gender  = seg.Segmentation(
    school_closures_3,
    [gender_segmentation]
).segmented_beta()

segmented_school_closures_4_gender  = seg.Segmentation(
    school_closures_4,
    [gender_segmentation]
).segmented_beta()

segmented_school_closures_5_gender  = seg.Segmentation(
    school_closures_5,
    [gender_segmentation]
).segmented_beta()

segmented_business_closures_2_gender  = seg.Segmentation(
    business_closures_2,
    [gender_segmentation]
).segmented_beta()

segmented_business_closures_3_gender  = seg.Segmentation(
    business_closures_3,
    [gender_segmentation]
).segmented_beta()

segmented_transit_2_gender  = seg.Segmentation(
    Transit_2,
    [gender_segmentation]
).segmented_beta()

segmented_number_of_infections_gender  = seg.Segmentation(
    number_of_infections,
    [gender_segmentation]
).segmented_beta()

segmented_healthcare_restrictions_2_gender  = seg.Segmentation(
    healthcare_restrictions_2,
    [gender_segmentation]
).segmented_beta()


segmented_healthcare_restrictions_3_gender  = seg.Segmentation(
    healthcare_restrictions_3,
    [gender_segmentation]
).segmented_beta()

# Hispanic segmentation
segmented_mask_mandates_2_Hispanic = seg.Segmentation(
    mask_mandates_2,
    [Hispanic_segmentation]
).segmented_beta()

segmented_mask_mandates_3_Hispanic = seg.Segmentation( 
    mask_mandates_3,
    [Hispanic_segmentation]
).segmented_beta()

segmented_mask_mandates_4_Hispanic = seg.Segmentation(
    mask_mandates_4,
    [Hispanic_segmentation]
).segmented_beta()

segmented_school_closures_2_Hispanic = seg.Segmentation(
    school_closures_2,
    [Hispanic_segmentation]
).segmented_beta()

segmented_school_closures_3_Hispanic = seg.Segmentation(
    school_closures_3,
    [Hispanic_segmentation]
).segmented_beta()

segmented_school_closures_4_Hispanic = seg.Segmentation(
    school_closures_4,
    [Hispanic_segmentation]
).segmented_beta()

segmented_school_closures_5_Hispanic = seg.Segmentation(
    school_closures_5,
    [Hispanic_segmentation]
).segmented_beta()

segmented_business_closures_2_Hispanic = seg.Segmentation(
    business_closures_2,
    [Hispanic_segmentation]
).segmented_beta()

segmented_business_closures_3_Hispanic = seg.Segmentation(
    business_closures_3,
    [Hispanic_segmentation]
).segmented_beta()

segmented_transit_2_Hispanic = seg.Segmentation(
    Transit_2,
    [Hispanic_segmentation]
).segmented_beta()

segmented_number_of_infections_Hispanic = seg.Segmentation(
    number_of_infections,
    [Hispanic_segmentation]
).segmented_beta()

segmented_healthcare_restrictions_2_Hispanic = seg.Segmentation(
    healthcare_restrictions_2,
    [Hispanic_segmentation]
).segmented_beta()

segmented_healthcare_restrictions_3_Hispanic = seg.Segmentation(
    healthcare_restrictions_3,
    [Hispanic_segmentation]
).segmented_beta()


# Child segmentation
segmented_mask_mandates_2_child = seg.Segmentation(
    mask_mandates_2,
    [child_segmentation]
).segmented_beta()

segmented_mask_mandates_3_child = seg.Segmentation( 
    mask_mandates_3,
    [child_segmentation]
).segmented_beta()

segmented_mask_mandates_4_child = seg.Segmentation(
    mask_mandates_4,
    [child_segmentation]
).segmented_beta()

segmented_school_closures_2_child = seg.Segmentation(
    school_closures_2,
    [child_segmentation]
).segmented_beta()

segmented_school_closures_3_child = seg.Segmentation(
    school_closures_3,
    [child_segmentation]
).segmented_beta()

segmented_school_closures_4_child = seg.Segmentation(
    school_closures_4,
    [child_segmentation]
).segmented_beta()

segmented_school_closures_5_child = seg.Segmentation(
    school_closures_5,
    [child_segmentation]
).segmented_beta()

segmented_business_closures_2_child = seg.Segmentation(
    business_closures_2,
    [child_segmentation]
).segmented_beta()

segmented_business_closures_3_child = seg.Segmentation(
    business_closures_3,
    [child_segmentation]
).segmented_beta()

segmented_transit_2_child = seg.Segmentation(
    Transit_2,
    [child_segmentation]
).segmented_beta()

segmented_number_of_infections_child = seg.Segmentation(
    number_of_infections,
    [child_segmentation]
).segmented_beta()

segmented_healthcare_restrictions_2_child = seg.Segmentation(
    healthcare_restrictions_2,
    [child_segmentation]
).segmented_beta()

segmented_healthcare_restrictions_3_child = seg.Segmentation(
    healthcare_restrictions_3,
    [child_segmentation]
).segmented_beta()


# Assisted Living segmentation
segmented_mask_mandates_2_assisted_living = seg.Segmentation(
    mask_mandates_2,
    [assisted_living_segmentation]
).segmented_beta()

segmented_mask_mandates_3_assisted_living = seg.Segmentation( 
    mask_mandates_3,
    [assisted_living_segmentation]
).segmented_beta()

segmented_mask_mandates_4_assisted_living = seg.Segmentation(
    mask_mandates_4,
    [assisted_living_segmentation]
).segmented_beta()

segmented_school_closures_2_assisted_living = seg.Segmentation(
    school_closures_2,
    [assisted_living_segmentation]
).segmented_beta()

segmented_school_closures_3_assisted_living = seg.Segmentation(
    school_closures_3,
    [assisted_living_segmentation]
).segmented_beta()

segmented_school_closures_4_assisted_living = seg.Segmentation(
    school_closures_4,
    [assisted_living_segmentation]
).segmented_beta()

segmented_school_closures_5_assisted_living = seg.Segmentation(
    school_closures_5,
    [assisted_living_segmentation]
).segmented_beta()

segmented_business_closures_2_assisted_living = seg.Segmentation(
    business_closures_2,
    [assisted_living_segmentation]
).segmented_beta()

segmented_business_closures_3_assisted_living = seg.Segmentation(
    business_closures_3,
    [assisted_living_segmentation]
).segmented_beta()

segmented_transit_2_assisted_living = seg.Segmentation(
    Transit_2,
    [assisted_living_segmentation]
).segmented_beta()

segmented_number_of_infections_assisted_living = seg.Segmentation(
    number_of_infections,
    [assisted_living_segmentation]
).segmented_beta()

segmented_healthcare_restrictions_2_assisted_living = seg.Segmentation(
    healthcare_restrictions_2,
    [assisted_living_segmentation]
).segmented_beta()

segmented_healthcare_restrictions_3_assisted_living = seg.Segmentation(
    healthcare_restrictions_3,
    [assisted_living_segmentation]
).segmented_beta()

# Vaccine segmentation
segmented_mask_mandates_2_vaccine = seg.Segmentation(
    mask_mandates_2,
    [vaccine_segmentation]
).segmented_beta()

segmented_mask_mandates_3_vaccine = seg.Segmentation( 
    mask_mandates_3,
    [vaccine_segmentation]
).segmented_beta()

segmented_mask_mandates_4_vaccine = seg.Segmentation(
    mask_mandates_4,
    [vaccine_segmentation]
).segmented_beta()

segmented_school_closures_2_vaccine = seg.Segmentation(
    school_closures_2,
    [vaccine_segmentation]
).segmented_beta()

segmented_school_closures_3_vaccine = seg.Segmentation(
    school_closures_3,
    [vaccine_segmentation]
).segmented_beta()

segmented_school_closures_4_vaccine = seg.Segmentation(
    school_closures_4,
    [vaccine_segmentation]
).segmented_beta()

segmented_school_closures_5_vaccine = seg.Segmentation(
    school_closures_5,
    [vaccine_segmentation]
).segmented_beta()

segmented_business_closures_2_vaccine = seg.Segmentation(
    business_closures_2,
    [vaccine_segmentation]
).segmented_beta()

segmented_business_closures_3_vaccine = seg.Segmentation(
    business_closures_3,
    [vaccine_segmentation]
).segmented_beta()

segmented_transit_2_vaccine = seg.Segmentation(
    Transit_2,
    [vaccine_segmentation]
).segmented_beta()

segmented_number_of_infections_vaccine = seg.Segmentation(
    number_of_infections,
    [vaccine_segmentation]
).segmented_beta()

segmented_healthcare_restrictions_2_vaccine = seg.Segmentation(
    healthcare_restrictions_2,
    [vaccine_segmentation]
).segmented_beta()

segmented_healthcare_restrictions_3_vaccine = seg.Segmentation(
    healthcare_restrictions_3,
    [vaccine_segmentation]
).segmented_beta()


# Chronic segmentation
segmented_mask_mandates_2_chronic = seg.Segmentation(
    mask_mandates_2,
    [chronic_segmentation]
).segmented_beta()

segmented_mask_mandates_3_chronic = seg.Segmentation( 
    mask_mandates_3,
    [chronic_segmentation]
).segmented_beta()

segmented_mask_mandates_4_chronic = seg.Segmentation(
    mask_mandates_4,
    [chronic_segmentation]
).segmented_beta()

segmented_school_closures_2_chronic = seg.Segmentation(
    school_closures_2,
    [chronic_segmentation]
).segmented_beta()

segmented_school_closures_3_chronic = seg.Segmentation(
    school_closures_3,
    [chronic_segmentation]
).segmented_beta()

segmented_school_closures_4_chronic = seg.Segmentation(
    school_closures_4,
    [chronic_segmentation]
).segmented_beta()

segmented_school_closures_5_chronic = seg.Segmentation(
    school_closures_5,
    [chronic_segmentation]
).segmented_beta()

segmented_business_closures_2_chronic = seg.Segmentation(
    business_closures_2,
    [chronic_segmentation]
).segmented_beta()

segmented_business_closures_3_chronic = seg.Segmentation(
    business_closures_3,
    [chronic_segmentation]
).segmented_beta()

segmented_transit_2_chronic = seg.Segmentation(
    Transit_2,
    [chronic_segmentation]
).segmented_beta()

segmented_number_of_infections_chronic = seg.Segmentation(
    number_of_infections,
    [chronic_segmentation]
).segmented_beta()

segmented_healthcare_restrictions_2_chronic = seg.Segmentation(
    healthcare_restrictions_2,
    [chronic_segmentation]
).segmented_beta()

segmented_healthcare_restrictions_3_chronic = seg.Segmentation(
    healthcare_restrictions_3,
    [chronic_segmentation]
).segmented_beta()


# Health Insurance segmentation
segmented_mask_mandates_2_health_insurance = seg.Segmentation(
    mask_mandates_2,
    [health_insurance_segmentation]
).segmented_beta()

segmented_mask_mandates_3_health_insurance = seg.Segmentation( 
    mask_mandates_3,
    [health_insurance_segmentation]
).segmented_beta()

segmented_mask_mandates_4_health_insurance = seg.Segmentation(
    mask_mandates_4,
    [health_insurance_segmentation]
).segmented_beta()

segmented_school_closures_2_health_insurance = seg.Segmentation(
    school_closures_2,
    [health_insurance_segmentation]
).segmented_beta()

segmented_school_closures_3_health_insurance = seg.Segmentation(
    school_closures_3,
    [health_insurance_segmentation]
).segmented_beta()

segmented_school_closures_4_health_insurance = seg.Segmentation(
    school_closures_4,
    [health_insurance_segmentation]
).segmented_beta()

segmented_school_closures_5_health_insurance = seg.Segmentation(
    school_closures_5,
    [health_insurance_segmentation]
).segmented_beta()

segmented_business_closures_2_health_insurance = seg.Segmentation(
    business_closures_2,
    [health_insurance_segmentation]
).segmented_beta()

segmented_business_closures_3_health_insurance = seg.Segmentation(
    business_closures_3,
    [health_insurance_segmentation]
).segmented_beta()

segmented_transit_2_health_insurance = seg.Segmentation(
    Transit_2,
    [health_insurance_segmentation]
).segmented_beta()

segmented_number_of_infections_health_insurance = seg.Segmentation(
    number_of_infections,
    [health_insurance_segmentation]
).segmented_beta()

segmented_healthcare_restrictions_2_health_insurance = seg.Segmentation(
    healthcare_restrictions_2,
    [health_insurance_segmentation]
).segmented_beta()

segmented_healthcare_restrictions_3_health_insurance = seg.Segmentation(
    healthcare_restrictions_3,
    [health_insurance_segmentation]
).segmented_beta()

# Self Employed segmentation
segmented_mask_mandates_2_self_employed = seg.Segmentation(
    mask_mandates_2,
    [self_employed_segmentation]
).segmented_beta()

segmented_mask_mandates_3_self_employed = seg.Segmentation( 
    mask_mandates_3,
    [self_employed_segmentation]
).segmented_beta()

segmented_mask_mandates_4_self_employed = seg.Segmentation(
    mask_mandates_4,
    [self_employed_segmentation]
).segmented_beta()

segmented_school_closures_2_self_employed = seg.Segmentation(
    school_closures_2,
    [self_employed_segmentation]
).segmented_beta()

segmented_school_closures_3_self_employed = seg.Segmentation(
    school_closures_3,
    [self_employed_segmentation]
).segmented_beta()

segmented_school_closures_4_self_employed = seg.Segmentation(
    school_closures_4,
    [self_employed_segmentation]
).segmented_beta()

segmented_school_closures_5_self_employed = seg.Segmentation(
    school_closures_5,
    [self_employed_segmentation]
).segmented_beta()

segmented_business_closures_2_self_employed = seg.Segmentation(
    business_closures_2,
    [self_employed_segmentation]
).segmented_beta()

segmented_business_closures_3_self_employed = seg.Segmentation(
    business_closures_3,
    [self_employed_segmentation]
).segmented_beta()

segmented_transit_2_self_employed = seg.Segmentation(
    Transit_2,
    [self_employed_segmentation]
).segmented_beta()

segmented_number_of_infections_self_employed = seg.Segmentation(
    number_of_infections,
    [self_employed_segmentation]
).segmented_beta()

segmented_healthcare_restrictions_2_self_employed = seg.Segmentation(
    healthcare_restrictions_2,
    [self_employed_segmentation]
).segmented_beta()

segmented_healthcare_restrictions_3_self_employed = seg.Segmentation(
    healthcare_restrictions_3,
    [self_employed_segmentation]
).segmented_beta()


# Remote segmentation
segmented_mask_mandates_2_remote = seg.Segmentation(
    mask_mandates_2,
    [remote_segmentation]
).segmented_beta()

segmented_mask_mandates_3_remote = seg.Segmentation( 
    mask_mandates_3,
    [remote_segmentation]
).segmented_beta()

segmented_mask_mandates_4_remote = seg.Segmentation(
    mask_mandates_4,
    [remote_segmentation]
).segmented_beta()

segmented_school_closures_2_remote = seg.Segmentation(
    school_closures_2,
    [remote_segmentation]
).segmented_beta()

segmented_school_closures_3_remote = seg.Segmentation(
    school_closures_3,
    [remote_segmentation]
).segmented_beta()

segmented_school_closures_4_remote = seg.Segmentation(
    school_closures_4,
    [remote_segmentation]
).segmented_beta()

segmented_school_closures_5_remote = seg.Segmentation(
    school_closures_5,
    [remote_segmentation]
).segmented_beta()

segmented_business_closures_2_remote = seg.Segmentation(
    business_closures_2,
    [remote_segmentation]
).segmented_beta()

segmented_business_closures_3_remote = seg.Segmentation(
    business_closures_3,
    [remote_segmentation]
).segmented_beta()

segmented_transit_2_remote = seg.Segmentation(
    Transit_2,
    [remote_segmentation]
).segmented_beta()

segmented_number_of_infections_remote = seg.Segmentation(
    number_of_infections,
    [remote_segmentation]
).segmented_beta()

segmented_healthcare_restrictions_2_remote = seg.Segmentation(
    healthcare_restrictions_2,
    [remote_segmentation]
).segmented_beta()

segmented_healthcare_restrictions_3_remote = seg.Segmentation(
    healthcare_restrictions_3,
    [remote_segmentation]
).segmented_beta()


# Pregnant segmentation
segmented_mask_mandates_2_pregnant = seg.Segmentation(
    mask_mandates_2,
    [pregnant_segmentation]
).segmented_beta()

segmented_mask_mandates_3_pregnant = seg.Segmentation( 
    mask_mandates_3,
    [pregnant_segmentation]
).segmented_beta()

segmented_mask_mandates_4_pregnant = seg.Segmentation(
    mask_mandates_4,
    [pregnant_segmentation]
).segmented_beta()

segmented_school_closures_2_pregnant = seg.Segmentation(
    school_closures_2,
    [pregnant_segmentation]
).segmented_beta()

segmented_school_closures_3_pregnant = seg.Segmentation(
    school_closures_3,
    [pregnant_segmentation]
).segmented_beta()

segmented_school_closures_4_pregnant = seg.Segmentation(
    school_closures_4,
    [pregnant_segmentation]
).segmented_beta()

segmented_school_closures_5_pregnant = seg.Segmentation(
    school_closures_5,
    [pregnant_segmentation]
).segmented_beta()

segmented_business_closures_2_pregnant = seg.Segmentation(
    business_closures_2,
    [pregnant_segmentation]
).segmented_beta()

segmented_business_closures_3_pregnant = seg.Segmentation(
    business_closures_3,
    [pregnant_segmentation]
).segmented_beta()

segmented_transit_2_pregnant = seg.Segmentation(
    Transit_2,
    [pregnant_segmentation]
).segmented_beta()

segmented_number_of_infections_pregnant = seg.Segmentation(
    number_of_infections,
    [pregnant_segmentation]
).segmented_beta()

segmented_healthcare_restrictions_2_pregnant = seg.Segmentation(
    healthcare_restrictions_2,
    [pregnant_segmentation]
).segmented_beta()

segmented_healthcare_restrictions_3_pregnant = seg.Segmentation(
    healthcare_restrictions_3,
    [pregnant_segmentation]
).segmented_beta()


# Vehicle segmentation
segmented_mask_mandates_2_vehicle = seg.Segmentation(
    mask_mandates_2,
    [vehicle_segmentation]
).segmented_beta()

segmented_mask_mandates_3_vehicle = seg.Segmentation( 
    mask_mandates_3,
    [vehicle_segmentation]
).segmented_beta()

segmented_mask_mandates_4_vehicle = seg.Segmentation(
    mask_mandates_4,
    [vehicle_segmentation]
).segmented_beta()

segmented_school_closures_2_vehicle = seg.Segmentation(
    school_closures_2,
    [vehicle_segmentation]
).segmented_beta()

segmented_school_closures_3_vehicle = seg.Segmentation(
    school_closures_3,
    [vehicle_segmentation]
).segmented_beta()

segmented_school_closures_4_vehicle = seg.Segmentation(
    school_closures_4,
    [vehicle_segmentation]
).segmented_beta()

segmented_school_closures_5_vehicle = seg.Segmentation(
    school_closures_5,
    [vehicle_segmentation]
).segmented_beta()

segmented_business_closures_2_vehicle = seg.Segmentation(
    business_closures_2,
    [vehicle_segmentation]
).segmented_beta()

segmented_business_closures_3_vehicle = seg.Segmentation(
    business_closures_3,
    [vehicle_segmentation]
).segmented_beta()

segmented_transit_2_vehicle = seg.Segmentation(
    Transit_2,
    [vehicle_segmentation]
).segmented_beta()

segmented_number_of_infections_vehicle = seg.Segmentation(
    number_of_infections,
    [vehicle_segmentation]
).segmented_beta()

segmented_healthcare_restrictions_2_vehicle = seg.Segmentation(
    healthcare_restrictions_2,
    [vehicle_segmentation]
).segmented_beta()

segmented_healthcare_restrictions_3_vehicle = seg.Segmentation(
    healthcare_restrictions_3,
    [vehicle_segmentation]
).segmented_beta()

segmented_mask_mandates_2_news = seg.Segmentation(
    mask_mandates_2,
    [news_segmentation]
).segmented_beta()

segmented_mask_mandates_3_news = seg.Segmentation( 
    mask_mandates_3,
    [news_segmentation]
).segmented_beta()

segmented_mask_mandates_4_news = seg.Segmentation(
    mask_mandates_4,
    [news_segmentation]
).segmented_beta()

segmented_school_closures_2_news = seg.Segmentation(
    school_closures_2,
    [news_segmentation]
).segmented_beta()

segmented_school_closures_3_news = seg.Segmentation(
    school_closures_3,
    [news_segmentation]
).segmented_beta()

segmented_school_closures_4_news = seg.Segmentation(
    school_closures_4,
    [news_segmentation]
).segmented_beta()

segmented_school_closures_5_news = seg.Segmentation(
    school_closures_5,
    [news_segmentation]
).segmented_beta()

segmented_business_closures_2_news = seg.Segmentation(
    business_closures_2,
    [news_segmentation]
).segmented_beta()

segmented_business_closures_3_news = seg.Segmentation(
    business_closures_3,
    [news_segmentation]
).segmented_beta()

segmented_transit_2_news = seg.Segmentation(
    Transit_2,
    [news_segmentation]
).segmented_beta()

segmented_number_of_infections_news = seg.Segmentation(
    number_of_infections,
    [news_segmentation]
).segmented_beta()

segmented_healthcare_restrictions_2_news = seg.Segmentation(
    healthcare_restrictions_2,
    [news_segmentation]
).segmented_beta()

segmented_healthcare_restrictions_3_news = seg.Segmentation(
    healthcare_restrictions_3,
    [news_segmentation]
).segmented_beta()

segmented_mask_mandates_2_political = seg.Segmentation(
    mask_mandates_2,
    [political_segmentation]
).segmented_beta()

segmented_mask_mandates_3_political = seg.Segmentation( 
    mask_mandates_3,
    [political_segmentation]
).segmented_beta()

segmented_mask_mandates_4_political = seg.Segmentation(
    mask_mandates_4,
    [political_segmentation]
).segmented_beta()

segmented_school_closures_2_political = seg.Segmentation(
    school_closures_2,
    [political_segmentation]
).segmented_beta()

segmented_school_closures_3_political = seg.Segmentation(
    school_closures_3,
    [political_segmentation]
).segmented_beta()

segmented_school_closures_4_political = seg.Segmentation(
    school_closures_4,
    [political_segmentation]
).segmented_beta()

segmented_school_closures_5_political = seg.Segmentation(
    school_closures_5,
    [political_segmentation]
).segmented_beta()

segmented_business_closures_2_political = seg.Segmentation(
    business_closures_2,
    [political_segmentation]
).segmented_beta()

segmented_business_closures_3_political = seg.Segmentation(
    business_closures_3,
    [political_segmentation]
).segmented_beta()

segmented_transit_2_political = seg.Segmentation(
    Transit_2,
    [political_segmentation]
).segmented_beta()

segmented_number_of_infections_political = seg.Segmentation(
    number_of_infections,
    [political_segmentation]
).segmented_beta()

segmented_healthcare_restrictions_2_political = seg.Segmentation(
    healthcare_restrictions_2,
    [political_segmentation]
).segmented_beta()

segmented_healthcare_restrictions_3_political = seg.Segmentation(
    healthcare_restrictions_3,
    [political_segmentation]
).segmented_beta()

# age segmentation
segmented_mask_mandates_2_age = seg.Segmentation(
    mask_mandates_2,
    [age_segmentation]
).segmented_beta()

segmented_mask_mandates_3_age = seg.Segmentation( 
    mask_mandates_3,
    [age_segmentation]
).segmented_beta()

segmented_mask_mandates_4_age = seg.Segmentation(
    mask_mandates_4,
    [age_segmentation]
).segmented_beta()

segmented_school_closures_2_age = seg.Segmentation(
    school_closures_2,
    [age_segmentation]
).segmented_beta()

segmented_school_closures_3_age = seg.Segmentation(
    school_closures_3,
    [age_segmentation]
).segmented_beta()

segmented_school_closures_4_age = seg.Segmentation(
    school_closures_4,
    [age_segmentation]
).segmented_beta()

segmented_school_closures_5_age = seg.Segmentation(
    school_closures_5,
    [age_segmentation]
).segmented_beta()

segmented_business_closures_2_age = seg.Segmentation(
    business_closures_2,
    [age_segmentation]
).segmented_beta()

segmented_business_closures_3_age = seg.Segmentation(
    business_closures_3,
    [age_segmentation]
).segmented_beta()

segmented_transit_2_age = seg.Segmentation(
    Transit_2,
    [age_segmentation]
).segmented_beta()

segmented_number_of_infections_age = seg.Segmentation(
    number_of_infections,
    [age_segmentation]
).segmented_beta()

segmented_healthcare_restrictions_2_age = seg.Segmentation(
    healthcare_restrictions_2,
    [age_segmentation]
).segmented_beta()

segmented_healthcare_restrictions_3_age = seg.Segmentation(
    healthcare_restrictions_3,
    [age_segmentation]
).segmented_beta()


# income segmentation
segmented_mask_mandates_2_income = seg.Segmentation(
    mask_mandates_2,
    [income_segmentation]
).segmented_beta()

segmented_mask_mandates_3_income = seg.Segmentation( 
    mask_mandates_3,
    [income_segmentation]
).segmented_beta()

segmented_mask_mandates_4_income = seg.Segmentation(
    mask_mandates_4,
    [income_segmentation]
).segmented_beta()

segmented_school_closures_2_income = seg.Segmentation(
    school_closures_2,
    [income_segmentation]
).segmented_beta()

segmented_school_closures_3_income = seg.Segmentation(
    school_closures_3,
    [income_segmentation]
).segmented_beta()

segmented_school_closures_4_income = seg.Segmentation(
    school_closures_4,
    [income_segmentation]
).segmented_beta()

segmented_school_closures_5_income = seg.Segmentation(
    school_closures_5,
    [income_segmentation]
).segmented_beta()

segmented_business_closures_2_income = seg.Segmentation(
    business_closures_2,
    [income_segmentation]
).segmented_beta()

segmented_business_closures_3_income = seg.Segmentation(
    business_closures_3,
    [income_segmentation]
).segmented_beta()

segmented_transit_2_income = seg.Segmentation(
    Transit_2,
    [income_segmentation]
).segmented_beta()

segmented_number_of_infections_income = seg.Segmentation(
    number_of_infections,
    [income_segmentation]
).segmented_beta()

segmented_healthcare_restrictions_2_income = seg.Segmentation(
    healthcare_restrictions_2,
    [income_segmentation]
).segmented_beta()

segmented_healthcare_restrictions_3_income = seg.Segmentation(
    healthcare_restrictions_3,
    [income_segmentation]
).segmented_beta()

# race segmentation
segmented_mask_mandates_2_race = seg.Segmentation(
    mask_mandates_2,
    [race_segmentation]
).segmented_beta()

segmented_mask_mandates_3_race = seg.Segmentation( 
    mask_mandates_3,
    [race_segmentation]
).segmented_beta()

segmented_mask_mandates_4_race = seg.Segmentation(
    mask_mandates_4,
    [race_segmentation]
).segmented_beta()

segmented_school_closures_2_race = seg.Segmentation(
    school_closures_2,
    [race_segmentation]
).segmented_beta()

segmented_school_closures_3_race = seg.Segmentation(
    school_closures_3,
    [race_segmentation]
).segmented_beta()

segmented_school_closures_4_race = seg.Segmentation(
    school_closures_4,
    [race_segmentation]
).segmented_beta()

segmented_school_closures_5_race = seg.Segmentation(
    school_closures_5,
    [race_segmentation]
).segmented_beta()

segmented_business_closures_2_race = seg.Segmentation(
    business_closures_2,
    [race_segmentation]
).segmented_beta()

segmented_business_closures_3_race = seg.Segmentation(
    business_closures_3,
    [race_segmentation]
).segmented_beta()

segmented_transit_2_race = seg.Segmentation(
    Transit_2,
    [race_segmentation]
).segmented_beta()

segmented_number_of_infections_race = seg.Segmentation(
    number_of_infections,
    [race_segmentation]
).segmented_beta()

segmented_healthcare_restrictions_2_race = seg.Segmentation(
    healthcare_restrictions_2,
    [race_segmentation]
).segmented_beta()

segmented_healthcare_restrictions_3_race = seg.Segmentation(
    healthcare_restrictions_3,
    [race_segmentation]
).segmented_beta()

# residence segmentation
segmented_mask_mandates_2_residence = seg.Segmentation(
    mask_mandates_2,
    [residence_segmentation]
).segmented_beta()

segmented_mask_mandates_3_residence = seg.Segmentation( 
    mask_mandates_3,
    [residence_segmentation]
).segmented_beta()

segmented_mask_mandates_4_residence = seg.Segmentation(
    mask_mandates_4,
    [residence_segmentation]
).segmented_beta()

segmented_school_closures_2_residence = seg.Segmentation(
    school_closures_2,
    [residence_segmentation]
).segmented_beta()

segmented_school_closures_3_residence = seg.Segmentation(
    school_closures_3,
    [residence_segmentation]
).segmented_beta()

segmented_school_closures_4_residence = seg.Segmentation(
    school_closures_4,
    [residence_segmentation]
).segmented_beta()

segmented_school_closures_5_residence = seg.Segmentation(
    school_closures_5,
    [residence_segmentation]
).segmented_beta()

segmented_business_closures_2_residence = seg.Segmentation(
    business_closures_2,
    [residence_segmentation]
).segmented_beta()

segmented_business_closures_3_residence = seg.Segmentation(
    business_closures_3,
    [residence_segmentation]
).segmented_beta()

segmented_transit_2_residence = seg.Segmentation(
    Transit_2,
    [residence_segmentation]
).segmented_beta()

segmented_number_of_infections_residence = seg.Segmentation(
    number_of_infections,
    [residence_segmentation]
).segmented_beta()

segmented_healthcare_restrictions_2_residence = seg.Segmentation(
    healthcare_restrictions_2,
    [residence_segmentation]
).segmented_beta()

segmented_healthcare_restrictions_3_residence = seg.Segmentation(
    healthcare_restrictions_3,
    [residence_segmentation]
).segmented_beta()

# vulnerable_contact segmentation
segmented_mask_mandates_2_vulnerable_contact = seg.Segmentation(
    mask_mandates_2,
    [vulnerable_contact_segmentation]
).segmented_beta()

segmented_mask_mandates_3_vulnerable_contact = seg.Segmentation( 
    mask_mandates_3,
    [vulnerable_contact_segmentation]
).segmented_beta()

segmented_mask_mandates_4_vulnerable_contact = seg.Segmentation(
    mask_mandates_4,
    [vulnerable_contact_segmentation]
).segmented_beta()

segmented_school_closures_2_vulnerable_contact = seg.Segmentation(
    school_closures_2,
    [vulnerable_contact_segmentation]
).segmented_beta()

segmented_school_closures_3_vulnerable_contact = seg.Segmentation(
    school_closures_3,
    [vulnerable_contact_segmentation]
).segmented_beta()

segmented_school_closures_4_vulnerable_contact = seg.Segmentation(
    school_closures_4,
    [vulnerable_contact_segmentation]
).segmented_beta()

segmented_school_closures_5_vulnerable_contact = seg.Segmentation(
    school_closures_5,
    [vulnerable_contact_segmentation]
).segmented_beta()

segmented_business_closures_2_vulnerable_contact = seg.Segmentation(
    business_closures_2,
    [vulnerable_contact_segmentation]
).segmented_beta()

segmented_business_closures_3_vulnerable_contact = seg.Segmentation(
    business_closures_3,
    [vulnerable_contact_segmentation]
).segmented_beta()

segmented_transit_2_vulnerable_contact = seg.Segmentation(
    Transit_2,
    [vulnerable_contact_segmentation]
).segmented_beta()

segmented_number_of_infections_vulnerable_contact = seg.Segmentation(
    number_of_infections,
    [vulnerable_contact_segmentation]
).segmented_beta()

segmented_healthcare_restrictions_2_vulnerable_contact = seg.Segmentation(
    healthcare_restrictions_2,
    [vulnerable_contact_segmentation]
).segmented_beta()

segmented_healthcare_restrictions_3_vulnerable_contact = seg.Segmentation(
    healthcare_restrictions_3,
    [vulnerable_contact_segmentation]
).segmented_beta()

# education segmentation
segmented_mask_mandates_2_education = seg.Segmentation(
    mask_mandates_2,
    [education_segmentation]
).segmented_beta()

segmented_mask_mandates_3_education = seg.Segmentation( 
    mask_mandates_3,
    [education_segmentation]
).segmented_beta()

segmented_mask_mandates_4_education = seg.Segmentation(
    mask_mandates_4,
    [education_segmentation]
).segmented_beta()

segmented_school_closures_2_education = seg.Segmentation(
    school_closures_2,
    [education_segmentation]
).segmented_beta()

segmented_school_closures_3_education = seg.Segmentation(
    school_closures_3,
    [education_segmentation]
).segmented_beta()

segmented_school_closures_4_education = seg.Segmentation(
    school_closures_4,
    [education_segmentation]
).segmented_beta()

segmented_school_closures_5_education = seg.Segmentation(
    school_closures_5,
    [education_segmentation]
).segmented_beta()

segmented_business_closures_2_education = seg.Segmentation(
    business_closures_2,
    [education_segmentation]
).segmented_beta()

segmented_business_closures_3_education = seg.Segmentation(
    business_closures_3,
    [education_segmentation]
).segmented_beta()

segmented_transit_2_education = seg.Segmentation(
    Transit_2,
    [education_segmentation]
).segmented_beta()

segmented_number_of_infections_education = seg.Segmentation(
    number_of_infections,
    [education_segmentation]
).segmented_beta()

segmented_healthcare_restrictions_2_education = seg.Segmentation(
    healthcare_restrictions_2,
    [education_segmentation]
).segmented_beta()

segmented_healthcare_restrictions_3_education = seg.Segmentation(
    healthcare_restrictions_3,
    [education_segmentation]
).segmented_beta()



Model based on gender

In [ ]:
# Utilities based on gender
Alt1 = [
    #mask_mandates_1 * Mask_mandates_1_1
     segmented_mask_mandates_2_gender * Variable(f'{t}_Mask_mandates_1_2')
    + segmented_mask_mandates_3_gender * Variable(f'{t}_Mask_mandates_1_3')
    + segmented_mask_mandates_4_gender * Variable(f'{t}_Mask_mandates_1_4')

    + segmented_school_closures_2_gender * Variable(f'{t}_School_closures_1_2')
    + segmented_school_closures_3_gender * Variable(f'{t}_School_closures_1_3')
    + segmented_school_closures_4_gender * Variable(f'{t}_School_closures_1_4')
    + segmented_school_closures_5_gender * Variable(f'{t}_School_closures_1_5')

    + segmented_business_closures_2_gender * Variable(f'{t}_Business_closures_1_2')
    + segmented_business_closures_3_gender * Variable(f'{t}_Business_closures_1_3')

    + segmented_transit_2_gender * Variable(f'{t}_Transit_1_2')

    + segmented_number_of_infections_gender* Variable(f'{t}_Number_infected_1')

    + segmented_healthcare_restrictions_2_gender * Variable(f'{t}_Healthcare_restrictions_1_2') 
    + segmented_healthcare_restrictions_3_gender * Variable(f'{t}_Healthcare_restrictions_1_3') 
     for t in range(1, 10)
]
Alt2 = [ACS2_param + 
    #mask_mandates_1 * Mask_mandates_2_1
     segmented_mask_mandates_2_gender * Variable(f'{t}_Mask_mandates_2_2')
    + segmented_mask_mandates_3_gender * Variable(f'{t}_Mask_mandates_2_3')
    + segmented_mask_mandates_4_gender * Variable(f'{t}_Mask_mandates_2_4')

    + segmented_school_closures_2_gender * Variable(f'{t}_School_closures_2_2')
    + segmented_school_closures_3_gender * Variable(f'{t}_School_closures_2_3')
    + segmented_school_closures_4_gender * Variable(f'{t}_School_closures_2_4')
    + segmented_school_closures_5_gender * Variable(f'{t}_School_closures_2_5')

    + segmented_business_closures_2_gender * Variable(f'{t}_Business_closures_2_2')
    + segmented_business_closures_3_gender * Variable(f'{t}_Business_closures_2_3')

    + segmented_transit_2_gender * Variable(f'{t}_Transit_2_2')

    + segmented_number_of_infections_gender * Variable(f'{t}_Number_infected_2')

    + segmented_healthcare_restrictions_2_gender * Variable(f'{t}_Healthcare_restrictions_2_2') 
    + segmented_healthcare_restrictions_3_gender * Variable(f'{t}_Healthcare_restrictions_2_3') 
    for t in range(1, 10)
]

# Associate utility functions with the numbering of alternatives
V = [{1: Alt1[t], 2: Alt2[t]} for t in range(9)]

# Associate the availability conditions with the alternatives

av = {1: 1, 2: 1}

obsprob = [loglogit(V[t], av, Variable(f'{t+1}_Chosen_Alternative')) for t in range(9)]
condprobIndiv = exp(bioMultSum(obsprob))

logprob = log(MonteCarlo(condprobIndiv))
the_biogeme = bio.BIOGEME(flat_database, logprob, parameter_file='few_draws.toml', numberOfDraws=500)
the_biogeme.modelName = 'b12panel_flat_gender_seg'

In [ ]:
results_gender = the_biogeme.estimate()
pandas_results_gender = results_gender.getEstimatedParameters(onlyRobust=False)
pandas_results_gender['CI_lower'] = pandas_results_gender['Value'] - pandas_results_gender['Std err'] * 1.96
pandas_results_gender['CI_upper'] = pandas_results_gender['Value'] + pandas_results_gender['Std err'] * 1.96
pandas_results_gender



Political affiliation

In [ ]:
#Utilities based on political affiliation
Alt1 = [
    #mask_mandates_1 * Mask_mandates_1_1
     segmented_mask_mandates_2_political * Variable(f'{t}_Mask_mandates_1_2')
    + segmented_mask_mandates_3_political * Variable(f'{t}_Mask_mandates_1_3')
    + segmented_mask_mandates_4_political * Variable(f'{t}_Mask_mandates_1_4')

    + segmented_school_closures_2_political * Variable(f'{t}_School_closures_1_2')
    + segmented_school_closures_3_political * Variable(f'{t}_School_closures_1_3')
    + segmented_school_closures_4_political * Variable(f'{t}_School_closures_1_4')
    + segmented_school_closures_5_political * Variable(f'{t}_School_closures_1_5')

    + segmented_business_closures_2_political * Variable(f'{t}_Business_closures_1_2')
    + segmented_business_closures_3_political * Variable(f'{t}_Business_closures_1_3')

    + segmented_transit_2_political * Variable(f'{t}_Transit_1_2')

    + segmented_number_of_infections_political* Variable(f'{t}_Number_infected_1')

    + segmented_healthcare_restrictions_2_political * Variable(f'{t}_Healthcare_restrictions_1_2') 
    + segmented_healthcare_restrictions_3_political * Variable(f'{t}_Healthcare_restrictions_1_3') 
     for t in range(1, 10)
]
Alt2 = [ACS2_param + 
    #mask_mandates_1 * Mask_mandates_2_1
     segmented_mask_mandates_2_political * Variable(f'{t}_Mask_mandates_2_2')
    + segmented_mask_mandates_3_political * Variable(f'{t}_Mask_mandates_2_3')
    + segmented_mask_mandates_4_political * Variable(f'{t}_Mask_mandates_2_4')

    + segmented_school_closures_2_political * Variable(f'{t}_School_closures_2_2')
    + segmented_school_closures_3_political * Variable(f'{t}_School_closures_2_3')
    + segmented_school_closures_4_political * Variable(f'{t}_School_closures_2_4')
    + segmented_school_closures_5_political * Variable(f'{t}_School_closures_2_5')

    + segmented_business_closures_2_political * Variable(f'{t}_Business_closures_2_2')
    + segmented_business_closures_3_political * Variable(f'{t}_Business_closures_2_3')

    + segmented_transit_2_political * Variable(f'{t}_Transit_2_2')

    + segmented_number_of_infections_political * Variable(f'{t}_Number_infected_2')

    + segmented_healthcare_restrictions_2_political * Variable(f'{t}_Healthcare_restrictions_2_2') 
    + segmented_healthcare_restrictions_3_political * Variable(f'{t}_Healthcare_restrictions_2_3') 
    for t in range(1, 10)
]

# Associate utility functions with the numbering of alternatives
V = [{1: Alt1[t], 2: Alt2[t]} for t in range(9)]

# Associate the availability conditions with the alternatives

av = {1: 1, 2: 1}

obsprob = [loglogit(V[t], av, Variable(f'{t+1}_Chosen_Alternative')) for t in range(9)]
condprobIndiv = exp(bioMultSum(obsprob))

logprob = log(MonteCarlo(condprobIndiv))
the_biogeme = bio.BIOGEME(flat_database, logprob, parameter_file='few_draws.toml', numberOfDraws=500)
the_biogeme.modelName = 'panel_flat_political_seg_pilot'


In [ ]:

results_political = the_biogeme.estimate()
results_political = results_political.getEstimatedParameters(onlyRobust=False)
results_political['CI_lower'] = results_political['Value'] - results_political['Std err'] * 1.96
results_political['CI_upper'] = results_political['Value'] + results_political['Std err'] * 1.96
results_political


In [ ]:
# age segmentation
segmented_mask_mandates_2_age = seg.Segmentation(
    mask_mandates_2,
    [age_segmentation]
).segmented_beta()

segmented_mask_mandates_3_age = seg.Segmentation( 
    mask_mandates_3,
    [age_segmentation]
).segmented_beta()

segmented_mask_mandates_4_age = seg.Segmentation(
    mask_mandates_4,
    [age_segmentation]
).segmented_beta()

segmented_school_closures_2_age = seg.Segmentation(
    school_closures_2,
    [age_segmentation]
).segmented_beta()

segmented_school_closures_3_age = seg.Segmentation(
    school_closures_3,
    [age_segmentation]
).segmented_beta()

segmented_school_closures_4_age = seg.Segmentation(
    school_closures_4,
    [age_segmentation]
).segmented_beta()

segmented_school_closures_5_age = seg.Segmentation(
    school_closures_5,
    [age_segmentation]
).segmented_beta()

segmented_business_closures_2_age = seg.Segmentation(
    business_closures_2,
    [age_segmentation]
).segmented_beta()

segmented_business_closures_3_age = seg.Segmentation(
    business_closures_3,
    [age_segmentation]
).segmented_beta()

segmented_transit_2_age = seg.Segmentation(
    Transit_2,
    [age_segmentation]
).segmented_beta()

segmented_number_of_infections_age = seg.Segmentation(
    number_of_infections,
    [age_segmentation]
).segmented_beta()

segmented_healthcare_restrictions_2_age = seg.Segmentation(
    healthcare_restrictions_2,
    [age_segmentation]
).segmented_beta()

segmented_healthcare_restrictions_3_age = seg.Segmentation(
    healthcare_restrictions_3,
    [age_segmentation]
).segmented_beta()


In [ ]:
#Utilities based on age affiliation
Alt1 = [
    #mask_mandates_1 * Mask_mandates_1_1
     segmented_mask_mandates_2_age * Variable(f'{t}_Mask_mandates_1_2')
    + segmented_mask_mandates_3_age * Variable(f'{t}_Mask_mandates_1_3')
    + segmented_mask_mandates_4_age * Variable(f'{t}_Mask_mandates_1_4')

    + segmented_school_closures_2_age * Variable(f'{t}_School_closures_1_2')
    + segmented_school_closures_3_age * Variable(f'{t}_School_closures_1_3')
    + segmented_school_closures_4_age * Variable(f'{t}_School_closures_1_4')
    + segmented_school_closures_5_age * Variable(f'{t}_School_closures_1_5')

    + segmented_business_closures_2_age * Variable(f'{t}_Business_closures_1_2')
    + segmented_business_closures_3_age * Variable(f'{t}_Business_closures_1_3')

    + segmented_transit_2_age * Variable(f'{t}_Transit_1_2')

    + segmented_number_of_infections_age* Variable(f'{t}_Number_infected_1')

    + segmented_healthcare_restrictions_2_age * Variable(f'{t}_Healthcare_restrictions_1_2') 
    + segmented_healthcare_restrictions_3_age * Variable(f'{t}_Healthcare_restrictions_1_3') 
     for t in range(1, 10)
]
Alt2 = [ACS2_param + 
    #mask_mandates_1 * Mask_mandates_2_1
     segmented_mask_mandates_2_age * Variable(f'{t}_Mask_mandates_2_2')
    + segmented_mask_mandates_3_age * Variable(f'{t}_Mask_mandates_2_3')
    + segmented_mask_mandates_4_age * Variable(f'{t}_Mask_mandates_2_4')

    + segmented_school_closures_2_age * Variable(f'{t}_School_closures_2_2')
    + segmented_school_closures_3_age * Variable(f'{t}_School_closures_2_3')
    + segmented_school_closures_4_age * Variable(f'{t}_School_closures_2_4')
    + segmented_school_closures_5_age * Variable(f'{t}_School_closures_2_5')

    + segmented_business_closures_2_age * Variable(f'{t}_Business_closures_2_2')
    + segmented_business_closures_3_age * Variable(f'{t}_Business_closures_2_3')

    + segmented_transit_2_age * Variable(f'{t}_Transit_2_2')

    + segmented_number_of_infections_age * Variable(f'{t}_Number_infected_2')

    + segmented_healthcare_restrictions_2_age * Variable(f'{t}_Healthcare_restrictions_2_2') 
    + segmented_healthcare_restrictions_3_age * Variable(f'{t}_Healthcare_restrictions_2_3') 
    for t in range(1, 10)
]

# Associate utility functions with the numbering of alternatives
V = [{1: Alt1[t], 2: Alt2[t]} for t in range(9)]

# Associate the availability conditions with the alternatives

av = {1: 1, 2: 1}

obsprob = [loglogit(V[t], av, Variable(f'{t+1}_Chosen_Alternative')) for t in range(9)]
condprobIndiv = exp(bioMultSum(obsprob))

logprob = log(MonteCarlo(condprobIndiv))
the_biogeme = bio.BIOGEME(flat_database, logprob, parameter_file='few_draws.toml', numberOfDraws=500)
the_biogeme.modelName = 'panel_flat_age_seg_pilot'


In [ ]:
results_age = the_biogeme.estimate()
results_age = results_age.getEstimatedParameters(onlyRobust=False)
results_age['CI_lower'] = results_age['Value'] - results_age['Std err'] * 1.96
results_age['CI_upper'] = results_age['Value'] + results_age['Std err'] * 1.96
results_age


Income

In [ ]:


#Utilities based on income affiliation
Alt1 = [
    #mask_mandates_1 * Mask_mandates_1_1
     segmented_mask_mandates_2_income * Variable(f'{t}_Mask_mandates_1_2')
    + segmented_mask_mandates_3_income * Variable(f'{t}_Mask_mandates_1_3')
    + segmented_mask_mandates_4_income * Variable(f'{t}_Mask_mandates_1_4')

    + segmented_school_closures_2_income * Variable(f'{t}_School_closures_1_2')
    + segmented_school_closures_3_income * Variable(f'{t}_School_closures_1_3')
    + segmented_school_closures_4_income * Variable(f'{t}_School_closures_1_4')
    + segmented_school_closures_5_income * Variable(f'{t}_School_closures_1_5')

    + segmented_business_closures_2_income * Variable(f'{t}_Business_closures_1_2')
    + segmented_business_closures_3_income * Variable(f'{t}_Business_closures_1_3')

    + segmented_transit_2_income * Variable(f'{t}_Transit_1_2')

    + segmented_number_of_infections_income* Variable(f'{t}_Number_infected_1')

    + segmented_healthcare_restrictions_2_income * Variable(f'{t}_Healthcare_restrictions_1_2') 
    + segmented_healthcare_restrictions_3_income * Variable(f'{t}_Healthcare_restrictions_1_3') 
     for t in range(1, 10)
]
Alt2 = [ACS2_param + 
    #mask_mandates_1 * Mask_mandates_2_1
     segmented_mask_mandates_2_income * Variable(f'{t}_Mask_mandates_2_2')
    + segmented_mask_mandates_3_income * Variable(f'{t}_Mask_mandates_2_3')
    + segmented_mask_mandates_4_income * Variable(f'{t}_Mask_mandates_2_4')

    + segmented_school_closures_2_income * Variable(f'{t}_School_closures_2_2')
    + segmented_school_closures_3_income * Variable(f'{t}_School_closures_2_3')
    + segmented_school_closures_4_income * Variable(f'{t}_School_closures_2_4')
    + segmented_school_closures_5_income * Variable(f'{t}_School_closures_2_5')

    + segmented_business_closures_2_income * Variable(f'{t}_Business_closures_2_2')
    + segmented_business_closures_3_income * Variable(f'{t}_Business_closures_2_3')

    + segmented_transit_2_income * Variable(f'{t}_Transit_2_2')

    + segmented_number_of_infections_income * Variable(f'{t}_Number_infected_2')

    + segmented_healthcare_restrictions_2_income * Variable(f'{t}_Healthcare_restrictions_2_2') 
    + segmented_healthcare_restrictions_3_income * Variable(f'{t}_Healthcare_restrictions_2_3') 
    for t in range(1, 10)
]

# Associate utility functions with the numbering of alternatives
V = [{1: Alt1[t], 2: Alt2[t]} for t in range(9)]

# Associate the availability conditions with the alternatives

av = {1: 1, 2: 1}

obsprob = [loglogit(V[t], av, Variable(f'{t+1}_Chosen_Alternative')) for t in range(9)]
condprobIndiv = exp(bioMultSum(obsprob))

logprob = log(MonteCarlo(condprobIndiv))
the_biogeme = bio.BIOGEME(flat_database, logprob, parameter_file='few_draws.toml', numberOfDraws=10)
the_biogeme.modelName = 'panel_flat_income_seg_pilot'



results_income = the_biogeme.estimate()
results_income = results_income.getEstimatedParameters(onlyRobust=False)
results_income['CI_lower'] = results_income['Value'] - results_income['Std err'] * 1.96
results_income['CI_upper'] = results_income['Value'] + results_income['Std err'] * 1.96
results_income


Chronic disease

In [ ]:


#Utilities based on chronic affiliation
Alt1 = [
    #mask_mandates_1 * Mask_mandates_1_1
     segmented_mask_mandates_2_chronic * Variable(f'{t}_Mask_mandates_1_2')
    + segmented_mask_mandates_3_chronic * Variable(f'{t}_Mask_mandates_1_3')
    + segmented_mask_mandates_4_chronic * Variable(f'{t}_Mask_mandates_1_4')

    + segmented_school_closures_2_chronic * Variable(f'{t}_School_closures_1_2')
    + segmented_school_closures_3_chronic * Variable(f'{t}_School_closures_1_3')
    + segmented_school_closures_4_chronic * Variable(f'{t}_School_closures_1_4')
    + segmented_school_closures_5_chronic * Variable(f'{t}_School_closures_1_5')

    + segmented_business_closures_2_chronic * Variable(f'{t}_Business_closures_1_2')
    + segmented_business_closures_3_chronic * Variable(f'{t}_Business_closures_1_3')

    + segmented_transit_2_chronic * Variable(f'{t}_Transit_1_2')

    + segmented_number_of_infections_chronic* Variable(f'{t}_Number_infected_1')

    + segmented_healthcare_restrictions_2_chronic * Variable(f'{t}_Healthcare_restrictions_1_2') 
    + segmented_healthcare_restrictions_3_chronic * Variable(f'{t}_Healthcare_restrictions_1_3') 
     for t in range(1, 10)
]
Alt2 = [ACS2_param + 
    #mask_mandates_1 * Mask_mandates_2_1
     segmented_mask_mandates_2_chronic * Variable(f'{t}_Mask_mandates_2_2')
    + segmented_mask_mandates_3_chronic * Variable(f'{t}_Mask_mandates_2_3')
    + segmented_mask_mandates_4_chronic * Variable(f'{t}_Mask_mandates_2_4')

    + segmented_school_closures_2_chronic * Variable(f'{t}_School_closures_2_2')
    + segmented_school_closures_3_chronic * Variable(f'{t}_School_closures_2_3')
    + segmented_school_closures_4_chronic * Variable(f'{t}_School_closures_2_4')
    + segmented_school_closures_5_chronic * Variable(f'{t}_School_closures_2_5')

    + segmented_business_closures_2_chronic * Variable(f'{t}_Business_closures_2_2')
    + segmented_business_closures_3_chronic * Variable(f'{t}_Business_closures_2_3')

    + segmented_transit_2_chronic * Variable(f'{t}_Transit_2_2')

    + segmented_number_of_infections_chronic * Variable(f'{t}_Number_infected_2')

    + segmented_healthcare_restrictions_2_chronic * Variable(f'{t}_Healthcare_restrictions_2_2') 
    + segmented_healthcare_restrictions_3_chronic * Variable(f'{t}_Healthcare_restrictions_2_3') 
    for t in range(1, 10)
]

# Associate utility functions with the numbering of alternatives
V = [{1: Alt1[t], 2: Alt2[t]} for t in range(9)]

# Associate the availability conditions with the alternatives

av = {1: 1, 2: 1}

obsprob = [loglogit(V[t], av, Variable(f'{t+1}_Chosen_Alternative')) for t in range(9)]
condprobIndiv = exp(bioMultSum(obsprob))

logprob = log(MonteCarlo(condprobIndiv))
the_biogeme = bio.BIOGEME(flat_database, logprob, parameter_file='few_draws.toml', numberOfDraws=10)
the_biogeme.modelName = 'panel_flat_chronic_seg_pilot'



results_chronic = the_biogeme.estimate()
results_chronic = results_chronic.getEstimatedParameters(onlyRobust=False)
results_chronic['CI_lower'] = results_chronic['Value'] - results_chronic['Std err'] * 1.96
results_chronic['CI_upper'] = results_chronic['Value'] + results_chronic['Std err'] * 1.96
results_chronic


Race

In [ ]:

#Utilities based on race affiliation
Alt1 = [
    #mask_mandates_1 * Mask_mandates_1_1
     segmented_mask_mandates_2_race * Variable(f'{t}_Mask_mandates_1_2')
    + segmented_mask_mandates_3_race * Variable(f'{t}_Mask_mandates_1_3')
    + segmented_mask_mandates_4_race * Variable(f'{t}_Mask_mandates_1_4')

    + segmented_school_closures_2_race * Variable(f'{t}_School_closures_1_2')
    + segmented_school_closures_3_race * Variable(f'{t}_School_closures_1_3')
    + segmented_school_closures_4_race * Variable(f'{t}_School_closures_1_4')
    + segmented_school_closures_5_race * Variable(f'{t}_School_closures_1_5')

    + segmented_business_closures_2_race * Variable(f'{t}_Business_closures_1_2')
    + segmented_business_closures_3_race * Variable(f'{t}_Business_closures_1_3')

    + segmented_transit_2_race * Variable(f'{t}_Transit_1_2')

    + segmented_number_of_infections_race* Variable(f'{t}_Number_infected_1')

    + segmented_healthcare_restrictions_2_race * Variable(f'{t}_Healthcare_restrictions_1_2') 
    + segmented_healthcare_restrictions_3_race * Variable(f'{t}_Healthcare_restrictions_1_3') 
     for t in range(1, 10)
]
Alt2 = [ACS2_param + 
    #mask_mandates_1 * Mask_mandates_2_1
     segmented_mask_mandates_2_race * Variable(f'{t}_Mask_mandates_2_2')
    + segmented_mask_mandates_3_race * Variable(f'{t}_Mask_mandates_2_3')
    + segmented_mask_mandates_4_race * Variable(f'{t}_Mask_mandates_2_4')

    + segmented_school_closures_2_race * Variable(f'{t}_School_closures_2_2')
    + segmented_school_closures_3_race * Variable(f'{t}_School_closures_2_3')
    + segmented_school_closures_4_race * Variable(f'{t}_School_closures_2_4')
    + segmented_school_closures_5_race * Variable(f'{t}_School_closures_2_5')

    + segmented_business_closures_2_race * Variable(f'{t}_Business_closures_2_2')
    + segmented_business_closures_3_race * Variable(f'{t}_Business_closures_2_3')

    + segmented_transit_2_race * Variable(f'{t}_Transit_2_2')

    + segmented_number_of_infections_race * Variable(f'{t}_Number_infected_2')

    + segmented_healthcare_restrictions_2_race * Variable(f'{t}_Healthcare_restrictions_2_2') 
    + segmented_healthcare_restrictions_3_race * Variable(f'{t}_Healthcare_restrictions_2_3') 
    for t in range(1, 10)
]

# Associate utility functions with the numbering of alternatives
V = [{1: Alt1[t], 2: Alt2[t]} for t in range(9)]

# Associate the availability conditions with the alternatives

av = {1: 1, 2: 1}

obsprob = [loglogit(V[t], av, Variable(f'{t+1}_Chosen_Alternative')) for t in range(9)]
condprobIndiv = exp(bioMultSum(obsprob))

logprob = log(MonteCarlo(condprobIndiv))
the_biogeme = bio.BIOGEME(flat_database, logprob, parameter_file='few_draws.toml', numberOfDraws=10)
the_biogeme.modelName = 'panel_flat_race_seg_pilot'



results_race = the_biogeme.estimate()
results_race = results_race.getEstimatedParameters(onlyRobust=False)
results_race['CI_lower'] = results_race['Value'] - results_race['Std err'] * 1.96
results_race['CI_upper'] = results_race['Value'] + results_race['Std err'] * 1.96
results_race

Vaccine status

In [ ]:


# Utilities based on vaccination affiliation
Alt1 = [
    segmented_mask_mandates_2_vaccine  * Variable(f'{t}_Mask_mandates_1_2')
    + segmented_mask_mandates_3_vaccine  * Variable(f'{t}_Mask_mandates_1_3')
    + segmented_mask_mandates_4_vaccine  * Variable(f'{t}_Mask_mandates_1_4')

    + segmented_school_closures_2_vaccine  * Variable(f'{t}_School_closures_1_2')
    + segmented_school_closures_3_vaccine  * Variable(f'{t}_School_closures_1_3')
    + segmented_school_closures_4_vaccine  * Variable(f'{t}_School_closures_1_4')
    + segmented_school_closures_5_vaccine * Variable(f'{t}_School_closures_1_5')

    + segmented_business_closures_2_vaccine  * Variable(f'{t}_Business_closures_1_2')
    + segmented_business_closures_3_vaccine  * Variable(f'{t}_Business_closures_1_3')

    + segmented_transit_2_vaccine * Variable(f'{t}_Transit_1_2')

    + segmented_number_of_infections_vaccine * Variable(f'{t}_Number_infected_1')

    + segmented_healthcare_restrictions_2_vaccine  * Variable(f'{t}_Healthcare_restrictions_1_2') 
    + segmented_healthcare_restrictions_3_vaccine  * Variable(f'{t}_Healthcare_restrictions_1_3') 
    for t in range(1, 10)
]

Alt2 = [ACS2_param + 
    segmented_mask_mandates_2_vaccine  * Variable(f'{t}_Mask_mandates_2_2')
    + segmented_mask_mandates_3_vaccine  * Variable(f'{t}_Mask_mandates_2_3')
    + segmented_mask_mandates_4_vaccine * Variable(f'{t}_Mask_mandates_2_4')

    + segmented_school_closures_2_vaccine * Variable(f'{t}_School_closures_2_2')
    + segmented_school_closures_3_vaccine  * Variable(f'{t}_School_closures_2_3')
    + segmented_school_closures_4_vaccine  * Variable(f'{t}_School_closures_2_4')
    + segmented_school_closures_5_vaccine * Variable(f'{t}_School_closures_2_5')

    + segmented_business_closures_2_vaccine  * Variable(f'{t}_Business_closures_2_2')
    + segmented_business_closures_3_vaccine * Variable(f'{t}_Business_closures_2_3')

    + segmented_transit_2_vaccine * Variable(f'{t}_Transit_2_2')

    + segmented_number_of_infections_vaccine  * Variable(f'{t}_Number_infected_2')

    + segmented_healthcare_restrictions_2_vaccine * Variable(f'{t}_Healthcare_restrictions_2_2') 
    + segmented_healthcare_restrictions_3_vaccine * Variable(f'{t}_Healthcare_restrictions_2_3') 
    for t in range(1, 10)
]

# Associate utility functions with the numbering of alternatives
V = [{1: Alt1[t], 2: Alt2[t]} for t in range(9)]

# Associate the availability conditions with the alternatives
av = {1: 1, 2: 1}

obsprob = [loglogit(V[t], av, Variable(f'{t+1}_Chosen_Alternative')) for t in range(9)]
condprobIndiv = exp(bioMultSum(obsprob))

logprob = log(MonteCarlo(condprobIndiv))
the_biogeme = bio.BIOGEME(flat_database, logprob, parameter_file='few_draws.toml', numberOfDraws=250)
the_biogeme.modelName = 'panel_flat_vaccination_seg_pilot'

results_vaccination = the_biogeme.estimate()
results_vaccination = results_vaccination.getEstimatedParameters(onlyRobust=False)
results_vaccination['CI_lower'] = results_vaccination['Value'] - results_vaccination['Std err'] * 1.96
results_vaccination['CI_upper'] = results_vaccination['Value'] + results_vaccination['Std err'] * 1.96
results_vaccination


Child 

In [ ]:

# Utilities based on child affiliation
Alt1 = [
    segmented_mask_mandates_2_child * Variable(f'{t}_Mask_mandates_1_2')
    + segmented_mask_mandates_3_child * Variable(f'{t}_Mask_mandates_1_3')
    + segmented_mask_mandates_4_child * Variable(f'{t}_Mask_mandates_1_4')

    + segmented_school_closures_2_child * Variable(f'{t}_School_closures_1_2')
    + segmented_school_closures_3_child * Variable(f'{t}_School_closures_1_3')
    + segmented_school_closures_4_child * Variable(f'{t}_School_closures_1_4')
    + segmented_school_closures_5_child * Variable(f'{t}_School_closures_1_5')

    + segmented_business_closures_2_child * Variable(f'{t}_Business_closures_1_2')
    + segmented_business_closures_3_child * Variable(f'{t}_Business_closures_1_3')

    + segmented_transit_2_child * Variable(f'{t}_Transit_1_2')

    + segmented_number_of_infections_child * Variable(f'{t}_Number_infected_1')

    + segmented_healthcare_restrictions_2_child * Variable(f'{t}_Healthcare_restrictions_1_2') 
    + segmented_healthcare_restrictions_3_child * Variable(f'{t}_Healthcare_restrictions_1_3') 
    for t in range(1, 10)
]

Alt2 = [ACS2_param + 
    segmented_mask_mandates_2_child * Variable(f'{t}_Mask_mandates_2_2')
    + segmented_mask_mandates_3_child * Variable(f'{t}_Mask_mandates_2_3')
    + segmented_mask_mandates_4_child * Variable(f'{t}_Mask_mandates_2_4')

    + segmented_school_closures_2_child * Variable(f'{t}_School_closures_2_2')
    + segmented_school_closures_3_child * Variable(f'{t}_School_closures_2_3')
    + segmented_school_closures_4_child * Variable(f'{t}_School_closures_2_4')
    + segmented_school_closures_5_child * Variable(f'{t}_School_closures_2_5')

    + segmented_business_closures_2_child * Variable(f'{t}_Business_closures_2_2')
    + segmented_business_closures_3_child * Variable(f'{t}_Business_closures_2_3')

    + segmented_transit_2_child * Variable(f'{t}_Transit_2_2')

    + segmented_number_of_infections_child * Variable(f'{t}_Number_infected_2')

    + segmented_healthcare_restrictions_2_child * Variable(f'{t}_Healthcare_restrictions_2_2') 
    + segmented_healthcare_restrictions_3_child * Variable(f'{t}_Healthcare_restrictions_2_3') 
    for t in range(1, 10)
]

# Associate utility functions with the numbering of alternatives
V = [{1: Alt1[t], 2: Alt2[t]} for t in range(9)]

# Associate the availability conditions with the alternatives
av = {1: 1, 2: 1}

obsprob = [loglogit(V[t], av, Variable(f'{t+1}_Chosen_Alternative')) for t in range(9)]
condprobIndiv = exp(bioMultSum(obsprob))

logprob = log(MonteCarlo(condprobIndiv))
the_biogeme = bio.BIOGEME(flat_database, logprob, parameter_file='few_draws.toml', numberOfDraws=500)
the_biogeme.modelName = 'panel_flat_child_seg_pilot'

results_child = the_biogeme.estimate()
results_child = results_child.getEstimatedParameters(onlyRobust=False)
results_child['CI_lower'] = results_child['Value'] - results_child['Std err'] * 1.96
results_child['CI_upper'] = results_child['Value'] + results_child['Std err'] * 1.96
results_child


Residence

In [ ]:


# Utilities based on residence affiliation
Alt1 = [
    segmented_mask_mandates_2_residence * Variable(f'{t}_Mask_mandates_1_2')
    + segmented_mask_mandates_3_residence * Variable(f'{t}_Mask_mandates_1_3')
    + segmented_mask_mandates_4_residence * Variable(f'{t}_Mask_mandates_1_4')

    + segmented_school_closures_2_residence * Variable(f'{t}_School_closures_1_2')
    + segmented_school_closures_3_residence * Variable(f'{t}_School_closures_1_3')
    + segmented_school_closures_4_residence * Variable(f'{t}_School_closures_1_4')
    + segmented_school_closures_5_residence * Variable(f'{t}_School_closures_1_5')

    + segmented_business_closures_2_residence * Variable(f'{t}_Business_closures_1_2')
    + segmented_business_closures_3_residence * Variable(f'{t}_Business_closures_1_3')

    + segmented_transit_2_residence * Variable(f'{t}_Transit_1_2')

    + segmented_number_of_infections_residence * Variable(f'{t}_Number_infected_1')

    + segmented_healthcare_restrictions_2_residence * Variable(f'{t}_Healthcare_restrictions_1_2') 
    + segmented_healthcare_restrictions_3_residence * Variable(f'{t}_Healthcare_restrictions_1_3') 
    for t in range(1, 10)
]

Alt2 = [ACS2_param + 
    segmented_mask_mandates_2_residence * Variable(f'{t}_Mask_mandates_2_2')
    + segmented_mask_mandates_3_residence * Variable(f'{t}_Mask_mandates_2_3')
    + segmented_mask_mandates_4_residence * Variable(f'{t}_Mask_mandates_2_4')

    + segmented_school_closures_2_residence * Variable(f'{t}_School_closures_2_2')
    + segmented_school_closures_3_residence * Variable(f'{t}_School_closures_2_3')
    + segmented_school_closures_4_residence * Variable(f'{t}_School_closures_2_4')
    + segmented_school_closures_5_residence * Variable(f'{t}_School_closures_2_5')

    + segmented_business_closures_2_residence * Variable(f'{t}_Business_closures_2_2')
    + segmented_business_closures_3_residence * Variable(f'{t}_Business_closures_2_3')

    + segmented_transit_2_residence * Variable(f'{t}_Transit_2_2')

    + segmented_number_of_infections_residence * Variable(f'{t}_Number_infected_2')

    + segmented_healthcare_restrictions_2_residence * Variable(f'{t}_Healthcare_restrictions_2_2') 
    + segmented_healthcare_restrictions_3_residence * Variable(f'{t}_Healthcare_restrictions_2_3') 
    for t in range(1, 10)
]

# Associate utility functions with the numbering of alternatives
V = [{1: Alt1[t], 2: Alt2[t]} for t in range(9)]

# Associate the availability conditions with the alternatives
av = {1: 1, 2: 1}

obsprob = [loglogit(V[t], av, Variable(f'{t+1}_Chosen_Alternative')) for t in range(9)]
condprobIndiv = exp(bioMultSum(obsprob))

logprob = log(MonteCarlo(condprobIndiv))
the_biogeme = bio.BIOGEME(flat_database, logprob, parameter_file='few_draws.toml', numberOfDraws=500)
the_biogeme.modelName = 'panel_flat_residence_seg_pilot'

results_residence = the_biogeme.estimate()
results_residence = results_residence.getEstimatedParameters(onlyRobust=False)
results_residence['CI_lower'] = results_residence['Value'] - results_residence['Std err'] * 1.96
results_residence['CI_upper'] = results_residence['Value'] + results_residence['Std err'] * 1.96
results_residence


Assisted Living

In [ ]:

# Utilities based on assisted_living affiliation
Alt1 = [
    segmented_mask_mandates_2_assisted_living * Variable(f'{t}_Mask_mandates_1_2')
    + segmented_mask_mandates_3_assisted_living * Variable(f'{t}_Mask_mandates_1_3')
    + segmented_mask_mandates_4_assisted_living * Variable(f'{t}_Mask_mandates_1_4')

    + segmented_school_closures_2_assisted_living * Variable(f'{t}_School_closures_1_2')
    + segmented_school_closures_3_assisted_living * Variable(f'{t}_School_closures_1_3')
    + segmented_school_closures_4_assisted_living * Variable(f'{t}_School_closures_1_4')
    + segmented_school_closures_5_assisted_living * Variable(f'{t}_School_closures_1_5')

    + segmented_business_closures_2_assisted_living * Variable(f'{t}_Business_closures_1_2')
    + segmented_business_closures_3_assisted_living * Variable(f'{t}_Business_closures_1_3')

    + segmented_transit_2_assisted_living * Variable(f'{t}_Transit_1_2')

    + segmented_number_of_infections_assisted_living * Variable(f'{t}_Number_infected_1')

    + segmented_healthcare_restrictions_2_assisted_living * Variable(f'{t}_Healthcare_restrictions_1_2') 
    + segmented_healthcare_restrictions_3_assisted_living * Variable(f'{t}_Healthcare_restrictions_1_3') 
    for t in range(1, 10)
]

Alt2 = [ACS2_param + 
    segmented_mask_mandates_2_assisted_living * Variable(f'{t}_Mask_mandates_2_2')
    + segmented_mask_mandates_3_assisted_living * Variable(f'{t}_Mask_mandates_2_3')
    + segmented_mask_mandates_4_assisted_living * Variable(f'{t}_Mask_mandates_2_4')

    + segmented_school_closures_2_assisted_living * Variable(f'{t}_School_closures_2_2')
    + segmented_school_closures_3_assisted_living * Variable(f'{t}_School_closures_2_3')
    + segmented_school_closures_4_assisted_living * Variable(f'{t}_School_closures_2_4')
    + segmented_school_closures_5_assisted_living * Variable(f'{t}_School_closures_2_5')

    + segmented_business_closures_2_assisted_living * Variable(f'{t}_Business_closures_2_2')
    + segmented_business_closures_3_assisted_living * Variable(f'{t}_Business_closures_2_3')

    + segmented_transit_2_assisted_living * Variable(f'{t}_Transit_2_2')

    + segmented_number_of_infections_assisted_living * Variable(f'{t}_Number_infected_2')

    + segmented_healthcare_restrictions_2_assisted_living * Variable(f'{t}_Healthcare_restrictions_2_2') 
    + segmented_healthcare_restrictions_3_assisted_living * Variable(f'{t}_Healthcare_restrictions_2_3') 
    for t in range(1, 10)
]

# Associate utility functions with the numbering of alternatives
V = [{1: Alt1[t], 2: Alt2[t]} for t in range(9)]

# Associate the availability conditions with the alternatives
av = {1: 1, 2: 1}

obsprob = [loglogit(V[t], av, Variable(f'{t+1}_Chosen_Alternative')) for t in range(9)]
condprobIndiv = exp(bioMultSum(obsprob))

logprob = log(MonteCarlo(condprobIndiv))
the_biogeme = bio.BIOGEME(flat_database, logprob, parameter_file='few_draws.toml', numberOfDraws=500)
the_biogeme.modelName = 'panel_flat_assisted_living_seg_pilot'

results_assisted_living = the_biogeme.estimate()
results_assisted_living = results_assisted_living.getEstimatedParameters(onlyRobust=False)
results_assisted_living['CI_lower'] = results_assisted_living['Value'] - results_assisted_living['Std err'] * 1.96
results_assisted_living['CI_upper'] = results_assisted_living['Value'] + results_assisted_living['Std err'] * 1.96
results_assisted_living


Vulnerable Contact

In [ ]:


# Utilities based on vulnerable_contact affiliation
Alt1 = [
    segmented_mask_mandates_2_vulnerable_contact * Variable(f'{t}_Mask_mandates_1_2')
    + segmented_mask_mandates_3_vulnerable_contact * Variable(f'{t}_Mask_mandates_1_3')
    + segmented_mask_mandates_4_vulnerable_contact * Variable(f'{t}_Mask_mandates_1_4')

    + segmented_school_closures_2_vulnerable_contact * Variable(f'{t}_School_closures_1_2')
    + segmented_school_closures_3_vulnerable_contact * Variable(f'{t}_School_closures_1_3')
    + segmented_school_closures_4_vulnerable_contact * Variable(f'{t}_School_closures_1_4')
    + segmented_school_closures_5_vulnerable_contact * Variable(f'{t}_School_closures_1_5')

    + segmented_business_closures_2_vulnerable_contact * Variable(f'{t}_Business_closures_1_2')
    + segmented_business_closures_3_vulnerable_contact * Variable(f'{t}_Business_closures_1_3')

    + segmented_transit_2_vulnerable_contact * Variable(f'{t}_Transit_1_2')

    + segmented_number_of_infections_vulnerable_contact * Variable(f'{t}_Number_infected_1')

    + segmented_healthcare_restrictions_2_vulnerable_contact * Variable(f'{t}_Healthcare_restrictions_1_2') 
    + segmented_healthcare_restrictions_3_vulnerable_contact * Variable(f'{t}_Healthcare_restrictions_1_3') 
    for t in range(1, 10)
]

Alt2 = [ACS2_param + 
    segmented_mask_mandates_2_vulnerable_contact * Variable(f'{t}_Mask_mandates_2_2')
    + segmented_mask_mandates_3_vulnerable_contact * Variable(f'{t}_Mask_mandates_2_3')
    + segmented_mask_mandates_4_vulnerable_contact * Variable(f'{t}_Mask_mandates_2_4')

    + segmented_school_closures_2_vulnerable_contact * Variable(f'{t}_School_closures_2_2')
    + segmented_school_closures_3_vulnerable_contact * Variable(f'{t}_School_closures_2_3')
    + segmented_school_closures_4_vulnerable_contact * Variable(f'{t}_School_closures_2_4')
    + segmented_school_closures_5_vulnerable_contact * Variable(f'{t}_School_closures_2_5')

    + segmented_business_closures_2_vulnerable_contact * Variable(f'{t}_Business_closures_2_2')
    + segmented_business_closures_3_vulnerable_contact * Variable(f'{t}_Business_closures_2_3')

    + segmented_transit_2_vulnerable_contact * Variable(f'{t}_Transit_2_2')

    + segmented_number_of_infections_vulnerable_contact * Variable(f'{t}_Number_infected_2')

    + segmented_healthcare_restrictions_2_vulnerable_contact * Variable(f'{t}_Healthcare_restrictions_2_2') 
    + segmented_healthcare_restrictions_3_vulnerable_contact * Variable(f'{t}_Healthcare_restrictions_2_3') 
    for t in range(1, 10)
]

# Associate utility functions with the numbering of alternatives
V = [{1: Alt1[t], 2: Alt2[t]} for t in range(9)]

# Associate the availability conditions with the alternatives
av = {1: 1, 2: 1}

obsprob = [loglogit(V[t], av, Variable(f'{t+1}_Chosen_Alternative')) for t in range(9)]
condprobIndiv = exp(bioMultSum(obsprob))

logprob = log(MonteCarlo(condprobIndiv))
the_biogeme = bio.BIOGEME(flat_database, logprob, parameter_file='few_draws.toml', numberOfDraws=500)
the_biogeme.modelName = 'panel_flat_vulnerable_contact_seg_pilot'

results_vulnerable_contact = the_biogeme.estimate()
results_vulnerable_contact = results_vulnerable_contact.getEstimatedParameters(onlyRobust=False)
results_vulnerable_contact['CI_lower'] = results_vulnerable_contact['Value'] - results_vulnerable_contact['Std err'] * 1.96
results_vulnerable_contact['CI_upper'] = results_vulnerable_contact['Value'] + results_vulnerable_contact['Std err'] * 1.96
results_vulnerable_contact


Health insurance 

In [ ]:

# Utilities based on health_insurance affiliation
Alt1 = [
    segmented_mask_mandates_2_health_insurance * Variable(f'{t}_Mask_mandates_1_2')
    + segmented_mask_mandates_3_health_insurance * Variable(f'{t}_Mask_mandates_1_3')
    + segmented_mask_mandates_4_health_insurance * Variable(f'{t}_Mask_mandates_1_4')

    + segmented_school_closures_2_health_insurance * Variable(f'{t}_School_closures_1_2')
    + segmented_school_closures_3_health_insurance * Variable(f'{t}_School_closures_1_3')
    + segmented_school_closures_4_health_insurance * Variable(f'{t}_School_closures_1_4')
    + segmented_school_closures_5_health_insurance * Variable(f'{t}_School_closures_1_5')

    + segmented_business_closures_2_health_insurance * Variable(f'{t}_Business_closures_1_2')
    + segmented_business_closures_3_health_insurance * Variable(f'{t}_Business_closures_1_3')

    + segmented_transit_2_health_insurance * Variable(f'{t}_Transit_1_2')

    + segmented_number_of_infections_health_insurance * Variable(f'{t}_Number_infected_1')

    + segmented_healthcare_restrictions_2_health_insurance * Variable(f'{t}_Healthcare_restrictions_1_2') 
    + segmented_healthcare_restrictions_3_health_insurance * Variable(f'{t}_Healthcare_restrictions_1_3') 
    for t in range(1, 10)
]

Alt2 = [ACS2_param + 
    segmented_mask_mandates_2_health_insurance * Variable(f'{t}_Mask_mandates_2_2')
    + segmented_mask_mandates_3_health_insurance * Variable(f'{t}_Mask_mandates_2_3')
    + segmented_mask_mandates_4_health_insurance * Variable(f'{t}_Mask_mandates_2_4')

    + segmented_school_closures_2_health_insurance * Variable(f'{t}_School_closures_2_2')
    + segmented_school_closures_3_health_insurance * Variable(f'{t}_School_closures_2_3')
    + segmented_school_closures_4_health_insurance * Variable(f'{t}_School_closures_2_4')
    + segmented_school_closures_5_health_insurance * Variable(f'{t}_School_closures_2_5')

    + segmented_business_closures_2_health_insurance * Variable(f'{t}_Business_closures_2_2')
    + segmented_business_closures_3_health_insurance * Variable(f'{t}_Business_closures_2_3')

    + segmented_transit_2_health_insurance * Variable(f'{t}_Transit_2_2')

    + segmented_number_of_infections_health_insurance * Variable(f'{t}_Number_infected_2')

    + segmented_healthcare_restrictions_2_health_insurance * Variable(f'{t}_Healthcare_restrictions_2_2') 
    + segmented_healthcare_restrictions_3_health_insurance * Variable(f'{t}_Healthcare_restrictions_2_3') 
    for t in range(1, 10)
]

# Associate utility functions with the numbering of alternatives
V = [{1: Alt1[t], 2: Alt2[t]} for t in range(9)]

# Associate the availability conditions with the alternatives
av = {1: 1, 2: 1}

obsprob = [loglogit(V[t], av, Variable(f'{t+1}_Chosen_Alternative')) for t in range(9)]
condprobIndiv = exp(bioMultSum(obsprob))

logprob = log(MonteCarlo(condprobIndiv))
the_biogeme = bio.BIOGEME(flat_database, logprob, parameter_file='few_draws.toml', numberOfDraws=500)
the_biogeme.modelName = 'panel_flat_health_insurance_seg_pilot'

results_health_insurance = the_biogeme.estimate()
results_health_insurance = results_health_insurance.getEstimatedParameters(onlyRobust=False)
results_health_insurance['CI_lower'] = results_health_insurance['Value'] - results_health_insurance['Std err'] * 1.96
results_health_insurance['CI_upper'] = results_health_insurance['Value'] + results_health_insurance['Std err'] * 1.96
results_health_insurance

Education

In [ ]:

# Utilities based on education affiliation
Alt1 = [
    segmented_mask_mandates_2_education * Variable(f'{t}_Mask_mandates_1_2')
    + segmented_mask_mandates_3_education * Variable(f'{t}_Mask_mandates_1_3')
    + segmented_mask_mandates_4_education * Variable(f'{t}_Mask_mandates_1_4')

    + segmented_school_closures_2_education * Variable(f'{t}_School_closures_1_2')
    + segmented_school_closures_3_education * Variable(f'{t}_School_closures_1_3')
    + segmented_school_closures_4_education * Variable(f'{t}_School_closures_1_4')
    + segmented_school_closures_5_education * Variable(f'{t}_School_closures_1_5')

    + segmented_business_closures_2_education * Variable(f'{t}_Business_closures_1_2')
    + segmented_business_closures_3_education * Variable(f'{t}_Business_closures_1_3')

    + segmented_transit_2_education * Variable(f'{t}_Transit_1_2')

    + segmented_number_of_infections_education * Variable(f'{t}_Number_infected_1')

    + segmented_healthcare_restrictions_2_education * Variable(f'{t}_Healthcare_restrictions_1_2') 
    + segmented_healthcare_restrictions_3_education * Variable(f'{t}_Healthcare_restrictions_1_3') 
    for t in range(1, 10)
]

Alt2 = [ACS2_param + 
    segmented_mask_mandates_2_education * Variable(f'{t}_Mask_mandates_2_2')
    + segmented_mask_mandates_3_education * Variable(f'{t}_Mask_mandates_2_3')
    + segmented_mask_mandates_4_education * Variable(f'{t}_Mask_mandates_2_4')

    + segmented_school_closures_2_education * Variable(f'{t}_School_closures_2_2')
    + segmented_school_closures_3_education * Variable(f'{t}_School_closures_2_3')
    + segmented_school_closures_4_education * Variable(f'{t}_School_closures_2_4')
    + segmented_school_closures_5_education * Variable(f'{t}_School_closures_2_5')

    + segmented_business_closures_2_education * Variable(f'{t}_Business_closures_2_2')
    + segmented_business_closures_3_education * Variable(f'{t}_Business_closures_2_3')

    + segmented_transit_2_education * Variable(f'{t}_Transit_2_2')

    + segmented_number_of_infections_education * Variable(f'{t}_Number_infected_2')

    + segmented_healthcare_restrictions_2_education * Variable(f'{t}_Healthcare_restrictions_2_2') 
    + segmented_healthcare_restrictions_3_education * Variable(f'{t}_Healthcare_restrictions_2_3') 
    for t in range(1, 10)
]

# Associate utility functions with the numbering of alternatives
V = [{1: Alt1[t], 2: Alt2[t]} for t in range(9)]

# Associate the availability conditions with the alternatives
av = {1: 1, 2: 1}

obsprob = [loglogit(V[t], av, Variable(f'{t+1}_Chosen_Alternative')) for t in range(9)]
condprobIndiv = exp(bioMultSum(obsprob))

logprob = log(MonteCarlo(condprobIndiv))
the_biogeme = bio.BIOGEME(flat_database, logprob, parameter_file='few_draws.toml', numberOfDraws=500)
the_biogeme.modelName = 'panel_flat_education_seg_pilot'

results_education = the_biogeme.estimate()
results_education = results_education.getEstimatedParameters(onlyRobust=False)
results_education['CI_lower'] = results_education['Value'] - results_education['Std err'] * 1.96
results_education['CI_upper'] = results_education['Value'] + results_education['Std err'] * 1.96
results_education


Pregnant

In [ ]:


# Utilities based on pregnant affiliation
Alt1 = [
    segmented_mask_mandates_2_pregnant * Variable(f'{t}_Mask_mandates_1_2')
    + segmented_mask_mandates_3_pregnant * Variable(f'{t}_Mask_mandates_1_3')
    + segmented_mask_mandates_4_pregnant * Variable(f'{t}_Mask_mandates_1_4')

    + segmented_school_closures_2_pregnant * Variable(f'{t}_School_closures_1_2')
    + segmented_school_closures_3_pregnant * Variable(f'{t}_School_closures_1_3')
    + segmented_school_closures_4_pregnant * Variable(f'{t}_School_closures_1_4')
    + segmented_school_closures_5_pregnant * Variable(f'{t}_School_closures_1_5')

    + segmented_business_closures_2_pregnant * Variable(f'{t}_Business_closures_1_2')
    + segmented_business_closures_3_pregnant * Variable(f'{t}_Business_closures_1_3')

    + segmented_transit_2_pregnant * Variable(f'{t}_Transit_1_2')

    + segmented_number_of_infections_pregnant * Variable(f'{t}_Number_infected_1')

    + segmented_healthcare_restrictions_2_pregnant * Variable(f'{t}_Healthcare_restrictions_1_2') 
    + segmented_healthcare_restrictions_3_pregnant * Variable(f'{t}_Healthcare_restrictions_1_3') 
    for t in range(1, 10)
]

Alt2 = [ACS2_param + 
    segmented_mask_mandates_2_pregnant * Variable(f'{t}_Mask_mandates_2_2')
    + segmented_mask_mandates_3_pregnant * Variable(f'{t}_Mask_mandates_2_3')
    + segmented_mask_mandates_4_pregnant * Variable(f'{t}_Mask_mandates_2_4')

    + segmented_school_closures_2_pregnant * Variable(f'{t}_School_closures_2_2')
    + segmented_school_closures_3_pregnant * Variable(f'{t}_School_closures_2_3')
    + segmented_school_closures_4_pregnant * Variable(f'{t}_School_closures_2_4')
    + segmented_school_closures_5_pregnant * Variable(f'{t}_School_closures_2_5')

    + segmented_business_closures_2_pregnant * Variable(f'{t}_Business_closures_2_2')
    + segmented_business_closures_3_pregnant * Variable(f'{t}_Business_closures_2_3')

    + segmented_transit_2_pregnant * Variable(f'{t}_Transit_2_2')

    + segmented_number_of_infections_pregnant * Variable(f'{t}_Number_infected_2')

    + segmented_healthcare_restrictions_2_pregnant * Variable(f'{t}_Healthcare_restrictions_2_2') 
    + segmented_healthcare_restrictions_3_pregnant * Variable(f'{t}_Healthcare_restrictions_2_3') 
    for t in range(1, 10)
]

# Associate utility functions with the numbering of alternatives
V = [{1: Alt1[t], 2: Alt2[t]} for t in range(9)]

# Associate the availability conditions with the alternatives
av = {1: 1, 2: 1}

obsprob = [loglogit(V[t], av, Variable(f'{t+1}_Chosen_Alternative')) for t in range(9)]
condprobIndiv = exp(bioMultSum(obsprob))

logprob = log(MonteCarlo(condprobIndiv))
the_biogeme = bio.BIOGEME(flat_database, logprob, parameter_file='few_draws.toml', numberOfDraws=500)
the_biogeme.modelName = 'panel_flat_pregnant_seg_pilot'

results_pregnant = the_biogeme.estimate()
results_pregnant = results_pregnant.getEstimatedParameters(onlyRobust=False)
results_pregnant['CI_lower'] = results_pregnant['Value'] - results_pregnant['Std err'] * 1.96
results_pregnant['CI_upper'] = results_pregnant['Value'] + results_pregnant['Std err'] * 1.96
results_pregnant


Vehicle

In [ ]:

# Utilities based on vehicle affiliation
Alt1 = [
    segmented_mask_mandates_2_vehicle * Variable(f'{t}_Mask_mandates_1_2')
    + segmented_mask_mandates_3_vehicle * Variable(f'{t}_Mask_mandates_1_3')
    + segmented_mask_mandates_4_vehicle * Variable(f'{t}_Mask_mandates_1_4')

    + segmented_school_closures_2_vehicle * Variable(f'{t}_School_closures_1_2')
    + segmented_school_closures_3_vehicle * Variable(f'{t}_School_closures_1_3')
    + segmented_school_closures_4_vehicle * Variable(f'{t}_School_closures_1_4')
    + segmented_school_closures_5_vehicle * Variable(f'{t}_School_closures_1_5')

    + segmented_business_closures_2_vehicle * Variable(f'{t}_Business_closures_1_2')
    + segmented_business_closures_3_vehicle * Variable(f'{t}_Business_closures_1_3')

    + segmented_transit_2_vehicle * Variable(f'{t}_Transit_1_2')

    + segmented_number_of_infections_vehicle * Variable(f'{t}_Number_infected_1')

    + segmented_healthcare_restrictions_2_vehicle * Variable(f'{t}_Healthcare_restrictions_1_2') 
    + segmented_healthcare_restrictions_3_vehicle * Variable(f'{t}_Healthcare_restrictions_1_3') 
    for t in range(1, 10)
]

Alt2 = [ACS2_param + 
    segmented_mask_mandates_2_vehicle * Variable(f'{t}_Mask_mandates_2_2')
    + segmented_mask_mandates_3_vehicle * Variable(f'{t}_Mask_mandates_2_3')
    + segmented_mask_mandates_4_vehicle * Variable(f'{t}_Mask_mandates_2_4')

    + segmented_school_closures_2_vehicle * Variable(f'{t}_School_closures_2_2')
    + segmented_school_closures_3_vehicle * Variable(f'{t}_School_closures_2_3')
    + segmented_school_closures_4_vehicle * Variable(f'{t}_School_closures_2_4')
    + segmented_school_closures_5_vehicle * Variable(f'{t}_School_closures_2_5')

    + segmented_business_closures_2_vehicle * Variable(f'{t}_Business_closures_2_2')
    + segmented_business_closures_3_vehicle * Variable(f'{t}_Business_closures_2_3')

    + segmented_transit_2_vehicle * Variable(f'{t}_Transit_2_2')

    + segmented_number_of_infections_vehicle * Variable(f'{t}_Number_infected_2')

    + segmented_healthcare_restrictions_2_vehicle * Variable(f'{t}_Healthcare_restrictions_2_2') 
    + segmented_healthcare_restrictions_3_vehicle * Variable(f'{t}_Healthcare_restrictions_2_3') 
    for t in range(1, 10)
]

# Associate utility functions with the numbering of alternatives
V = [{1: Alt1[t], 2: Alt2[t]} for t in range(9)]

# Associate the availability conditions with the alternatives
av = {1: 1, 2: 1}

obsprob = [loglogit(V[t], av, Variable(f'{t+1}_Chosen_Alternative')) for t in range(9)]
condprobIndiv = exp(bioMultSum(obsprob))

logprob = log(MonteCarlo(condprobIndiv))
the_biogeme = bio.BIOGEME(flat_database, logprob, parameter_file='few_draws.toml', numberOfDraws=500)
the_biogeme.modelName = 'panel_flat_vehicle_seg_pilot'

results_vehicle = the_biogeme.estimate()
results_vehicle = results_vehicle.getEstimatedParameters(onlyRobust=False)
results_vehicle['CI_lower'] = results_vehicle['Value'] - results_vehicle['Std err'] * 1.96
results_vehicle['CI_upper'] = results_vehicle['Value'] + results_vehicle['Std err'] * 1.96
results_vehicle


Remote

In [ ]:
# Utilities based on remote affiliation
Alt1 = [
    segmented_mask_mandates_2_remote * Variable(f'{t}_Mask_mandates_1_2')
    + segmented_mask_mandates_3_remote * Variable(f'{t}_Mask_mandates_1_3')
    + segmented_mask_mandates_4_remote * Variable(f'{t}_Mask_mandates_1_4')

    + segmented_school_closures_2_remote * Variable(f'{t}_School_closures_1_2')
    + segmented_school_closures_3_remote * Variable(f'{t}_School_closures_1_3')
    + segmented_school_closures_4_remote * Variable(f'{t}_School_closures_1_4')
    + segmented_school_closures_5_remote * Variable(f'{t}_School_closures_1_5')

    + segmented_business_closures_2_remote * Variable(f'{t}_Business_closures_1_2')
    + segmented_business_closures_3_remote * Variable(f'{t}_Business_closures_1_3')

    + segmented_transit_2_remote * Variable(f'{t}_Transit_1_2')

    + segmented_number_of_infections_remote * Variable(f'{t}_Number_infected_1')

    + segmented_healthcare_restrictions_2_remote * Variable(f'{t}_Healthcare_restrictions_1_2') 
    + segmented_healthcare_restrictions_3_remote * Variable(f'{t}_Healthcare_restrictions_1_3') 
    for t in range(1, 10)
]

Alt2 = [ACS2_param + 
    segmented_mask_mandates_2_remote * Variable(f'{t}_Mask_mandates_2_2')
    + segmented_mask_mandates_3_remote * Variable(f'{t}_Mask_mandates_2_3')
    + segmented_mask_mandates_4_remote * Variable(f'{t}_Mask_mandates_2_4')

    + segmented_school_closures_2_remote * Variable(f'{t}_School_closures_2_2')
    + segmented_school_closures_3_remote * Variable(f'{t}_School_closures_2_3')
    + segmented_school_closures_4_remote * Variable(f'{t}_School_closures_2_4')
    + segmented_school_closures_5_remote * Variable(f'{t}_School_closures_2_5')

    + segmented_business_closures_2_remote * Variable(f'{t}_Business_closures_2_2')
    + segmented_business_closures_3_remote * Variable(f'{t}_Business_closures_2_3')

    + segmented_transit_2_remote * Variable(f'{t}_Transit_2_2')

    + segmented_number_of_infections_remote * Variable(f'{t}_Number_infected_2')

    + segmented_healthcare_restrictions_2_remote * Variable(f'{t}_Healthcare_restrictions_2_2') 
    + segmented_healthcare_restrictions_3_remote * Variable(f'{t}_Healthcare_restrictions_2_3') 
    for t in range(1, 10)
]

# Associate utility functions with the numbering of alternatives
V = [{1: Alt1[t], 2: Alt2[t]} for t in range(9)]

# Associate the availability conditions with the alternatives
av = {1: 1, 2: 1}

obsprob = [loglogit(V[t], av, Variable(f'{t+1}_Chosen_Alternative')) for t in range(9)]
condprobIndiv = exp(bioMultSum(obsprob))

logprob = log(MonteCarlo(condprobIndiv))
the_biogeme = bio.BIOGEME(flat_database, logprob, parameter_file='few_draws.toml', numberOfDraws=500)
the_biogeme.modelName = 'panel_flat_remote_seg_pilot'

results_remote = the_biogeme.estimate()
results_remote = results_remote.getEstimatedParameters(onlyRobust=False)
results_remote['CI_lower'] = results_remote['Value'] - results_remote['Std err'] * 1.96
results_remote['CI_upper'] = results_remote['Value'] + results_remote['Std err'] * 1.96
results_remote

Self employed

In [ ]:


# Utilities based on self_employed affiliation
Alt1 = [
    segmented_mask_mandates_2_self_employed * Variable(f'{t}_Mask_mandates_1_2')
    + segmented_mask_mandates_3_self_employed * Variable(f'{t}_Mask_mandates_1_3')
    + segmented_mask_mandates_4_self_employed * Variable(f'{t}_Mask_mandates_1_4')

    + segmented_school_closures_2_self_employed * Variable(f'{t}_School_closures_1_2')
    + segmented_school_closures_3_self_employed * Variable(f'{t}_School_closures_1_3')
    + segmented_school_closures_4_self_employed * Variable(f'{t}_School_closures_1_4')
    + segmented_school_closures_5_self_employed * Variable(f'{t}_School_closures_1_5')

    + segmented_business_closures_2_self_employed * Variable(f'{t}_Business_closures_1_2')
    + segmented_business_closures_3_self_employed * Variable(f'{t}_Business_closures_1_3')

    + segmented_transit_2_self_employed * Variable(f'{t}_Transit_1_2')

    + segmented_number_of_infections_self_employed * Variable(f'{t}_Number_infected_1')

    + segmented_healthcare_restrictions_2_self_employed * Variable(f'{t}_Healthcare_restrictions_1_2') 
    + segmented_healthcare_restrictions_3_self_employed * Variable(f'{t}_Healthcare_restrictions_1_3') 
    for t in range(1, 10)
]

Alt2 = [ACS2_param + 
    segmented_mask_mandates_2_self_employed * Variable(f'{t}_Mask_mandates_2_2')
    + segmented_mask_mandates_3_self_employed * Variable(f'{t}_Mask_mandates_2_3')
    + segmented_mask_mandates_4_self_employed * Variable(f'{t}_Mask_mandates_2_4')

    + segmented_school_closures_2_self_employed * Variable(f'{t}_School_closures_2_2')
    + segmented_school_closures_3_self_employed * Variable(f'{t}_School_closures_2_3')
    + segmented_school_closures_4_self_employed * Variable(f'{t}_School_closures_2_4')
    + segmented_school_closures_5_self_employed * Variable(f'{t}_School_closures_2_5')

    + segmented_business_closures_2_self_employed * Variable(f'{t}_Business_closures_2_2')
    + segmented_business_closures_3_self_employed * Variable(f'{t}_Business_closures_2_3')

    + segmented_transit_2_self_employed * Variable(f'{t}_Transit_2_2')

    + segmented_number_of_infections_self_employed * Variable(f'{t}_Number_infected_2')

    + segmented_healthcare_restrictions_2_self_employed * Variable(f'{t}_Healthcare_restrictions_2_2') 
    + segmented_healthcare_restrictions_3_self_employed * Variable(f'{t}_Healthcare_restrictions_2_3') 
    for t in range(1, 10)
]

# Associate utility functions with the numbering of alternatives
V = [{1: Alt1[t], 2: Alt2[t]} for t in range(9)]

# Associate the availability conditions with the alternatives
av = {1: 1, 2: 1}

obsprob = [loglogit(V[t], av, Variable(f'{t+1}_Chosen_Alternative')) for t in range(9)]
condprobIndiv = exp(bioMultSum(obsprob))

logprob = log(MonteCarlo(condprobIndiv))
the_biogeme = bio.BIOGEME(flat_database, logprob, parameter_file='few_draws.toml', numberOfDraws=500)
the_biogeme.modelName = 'panel_flat_self_employed_seg_pilot'

results_self_employed = the_biogeme.estimate()
results_self_employed = results_self_employed.getEstimatedParameters(onlyRobust=False)
results_self_employed['CI_lower'] = results_self_employed['Value'] - results_self_employed['Std err'] * 1.96
results_self_employed['CI_upper'] = results_self_employed['Value'] + results_self_employed['Std err'] * 1.96
results_self_employed


News

In [ ]:
# Utilities based on news affiliation
Alt1 = [
    segmented_mask_mandates_2_news * Variable(f'{t}_Mask_mandates_1_2')
    + segmented_mask_mandates_3_news * Variable(f'{t}_Mask_mandates_1_3')
    + segmented_mask_mandates_4_news * Variable(f'{t}_Mask_mandates_1_4')

    + segmented_school_closures_2_news * Variable(f'{t}_School_closures_1_2')
    + segmented_school_closures_3_news * Variable(f'{t}_School_closures_1_3')
    + segmented_school_closures_4_news * Variable(f'{t}_School_closures_1_4')
    + segmented_school_closures_5_news * Variable(f'{t}_School_closures_1_5')

    + segmented_business_closures_2_news * Variable(f'{t}_Business_closures_1_2')
    + segmented_business_closures_3_news * Variable(f'{t}_Business_closures_1_3')

    + segmented_transit_2_news * Variable(f'{t}_Transit_1_2')

    + segmented_number_of_infections_news * Variable(f'{t}_Number_infected_1')

    + segmented_healthcare_restrictions_2_news * Variable(f'{t}_Healthcare_restrictions_1_2') 
    + segmented_healthcare_restrictions_3_news * Variable(f'{t}_Healthcare_restrictions_1_3') 
    for t in range(1, 10)
]

Alt2 = [ACS2_param + 
    segmented_mask_mandates_2_news * Variable(f'{t}_Mask_mandates_2_2')
    + segmented_mask_mandates_3_news * Variable(f'{t}_Mask_mandates_2_3')
    + segmented_mask_mandates_4_news * Variable(f'{t}_Mask_mandates_2_4')

    + segmented_school_closures_2_news * Variable(f'{t}_School_closures_2_2')
    + segmented_school_closures_3_news * Variable(f'{t}_School_closures_2_3')
    + segmented_school_closures_4_news * Variable(f'{t}_School_closures_2_4')
    + segmented_school_closures_5_news * Variable(f'{t}_School_closures_2_5')

    + segmented_business_closures_2_news * Variable(f'{t}_Business_closures_2_2')
    + segmented_business_closures_3_news * Variable(f'{t}_Business_closures_2_3')

    + segmented_transit_2_news * Variable(f'{t}_Transit_2_2')

    + segmented_number_of_infections_news * Variable(f'{t}_Number_infected_2')

    + segmented_healthcare_restrictions_2_news * Variable(f'{t}_Healthcare_restrictions_2_2') 
    + segmented_healthcare_restrictions_3_news * Variable(f'{t}_Healthcare_restrictions_2_3') 
    for t in range(1, 10)
]

# Associate utility functions with the numbering of alternatives
V = [{1: Alt1[t], 2: Alt2[t]} for t in range(9)]

# Associate the availability conditions with the alternatives
av = {1: 1, 2: 1}

obsprob = [loglogit(V[t], av, Variable(f'{t+1}_Chosen_Alternative')) for t in range(9)]
condprobIndiv = exp(bioMultSum(obsprob))

logprob = log(MonteCarlo(condprobIndiv))
the_biogeme = bio.BIOGEME(flat_database, logprob, parameter_file='few_draws.toml', numberOfDraws=500)
the_biogeme.modelName = 'panel_flat_news_seg_pilot'

results_news = the_biogeme.estimate()
results_news = results_news.getEstimatedParameters(onlyRobust=False)
results_news['CI_lower'] = results_news['Value'] - results_news['Std err'] * 1.96
results_news['CI_upper'] = results_news['Value'] + results_news['Std err'] * 1.96
results_news


# Willingness to pay 

General

In [ ]:
# Function to simulate draws from the estimated distribution of WTP
def simulate_wtp_draws(attribute_mean, attribute_std_dev, infection_mean, infection_std_dev, num_draws):
    # Draw from the estimated distribution of the attribute's coefficient
    wtp_draws = []
    for i in range(1000):
        attribute_coefficient_draws_mean = np.mean(np.random.normal(attribute_mean , attribute_std_dev, num_draws))
        # Draw from the estimated distribution of the price coefficient
        price_coefficient_draws_mean = np.mean(np.random.normal(infection_mean, infection_std_dev, num_draws))    # Calculate WTP draws
        wtp_draws.append( attribute_coefficient_draws_mean / price_coefficient_draws_mean)

    return wtp_draws

num_draws = 100000

In [ ]:
def calculate_wta_and_simulate(
        attribute_mean, attribute_std_err,
        infection_mean, infection_std_err,
        num_draws = 1000, alpha=0.05):

    # Step 1: Calculate WTA (or WTP) as the ratio of means
    hat_W = attribute_mean / infection_mean
    # Step 2: Simulate WTA/WTP draws and calculate confidence intervals
    wtp_differences = []
    # Repeat the process N times
    for i in range(num_draws):
        # Draw 1000 samples from N(attribute_mean, attribute_std_err)
        attribute_samples = np.random.normal(attribute_mean, attribute_std_err, 1)
        X_star_n = np.mean(attribute_samples)  # Take the average of the samples
        # Draw 1000 samples from N(infection_mean, infection_std_err)
        infection_samples = np.random.normal(infection_mean, infection_std_err, 1)
        Y_star_n = np.mean(infection_samples)  # Take the average of the samples
        # Calculate the difference: X_star_n / Y_star_n - hat_W
        d_star_n = (X_star_n / Y_star_n) - hat_W
        wtp_differences.append(d_star_n)
    # Calculate the percentiles for confidence intervals
    delta_alpha_2 = np.percentile(wtp_differences, (1 - alpha/2) * 100)
    delta_1_minus_alpha_2 = np.percentile(wtp_differences, (alpha/2) * 100)
    # Return the confidence interval
    lower_bound = hat_W - delta_alpha_2
    upper_bound = hat_W - delta_1_minus_alpha_2
    return hat_W, lower_bound, upper_bound

num_draws = 100000
n  = num_participants
# Retrieve the mean and std dev of the price coefficient
infection_index = results_drop_first_level.index.get_loc('Number_of_infections')
infection_mean = results_drop_first_level.iloc[infection_index]['Value']
infection_std_dev = results_drop_first_level.iloc[infection_index]['Std err']   

# Iterate over DataFrame rows and simulate WTP draws
for index, row in results_drop_first_level.iterrows():
    attribute_mean = row["Value"]
    attribute_std_dev = row["Std err"]   
    
    # Skip Price attribute
    if index != infection_index:
        wtp_mean, wtp_CI_lower, wtp_CI_upper = calculate_wta_and_simulate(attribute_mean, attribute_std_dev, infection_mean, infection_std_dev, num_draws)


        # Add results to the appropriate row in the DataFrame
        results_drop_first_level.loc[index, 'WTP_mean'] = wtp_mean
        #pandas_results_drop_first_level.loc[index, 'WTP_ste'] = 1.96 * np.std(wtp_draws) / np.sqrt(n)
        results_drop_first_level.loc[index, 'WTP_CI_lower'] = wtp_CI_lower
        results_drop_first_level.loc[index, 'WTP_CI_upper'] = wtp_CI_upper


        print(f"Attribute: {index}")
        print(f"Estimated mean of WTP: {wtp_mean}")
        print(f"Estimated lower CI of WTP: {wtp_CI_lower}")
        print(f"Estimated upper CI of WTP: {wtp_CI_upper}")


Gender segmentation

In [ ]:
num_draws = 100000
n  = num_participants
# Retrieve the mean and std dev of the price coefficient
infection_index_F = results_gender.index.get_loc('Number_of_infections')
infection_mean_F = results_gender.iloc[infection_index_F]['Value']
infection_std_dev_F = results_gender.iloc[infection_index_F]['Std err']

infection_index_M = results_gender.index.get_loc('Number_of_infections_Male')
infection_mean_M = results_gender.iloc[infection_index_M]['Value']
infection_std_dev_M = results_gender.iloc[infection_index_M]['Std err']

infection_index_NB = results_gender.index.get_loc('Number_of_infections_Non-binary or Self-define or Prefer not to answer')
infection_mean_NB = results_gender.iloc[infection_index_NB]['Value']
infection_std_dev_NB = results_gender.iloc[infection_index_NB]['Std err']


# Iterate over DataFrame rows and simulate WTP draws
for index, row in results_gender.iterrows():
    if "Male" in index:
        infection_mean = infection_mean_M
        infection_std_dev = infection_std_dev_M
        infection_index = infection_index_M
    elif "Non-binary" in index:
        infection_mean = infection_mean_NB
        infection_std_dev = infection_std_dev_NB
        infection_index = infection_index_NB
    else:   
        infection_mean = infection_mean_F
        infection_std_dev = infection_std_dev_F
        infection_index = infection_index_F
    attribute_mean = row["Value"]
    attribute_std_dev = row["Std err"]   
    
    # Skip Price attribute
    if index != infection_index:
        wtp_mean, wtp_CI_lower, wtp_CI_upper = calculate_wta_and_simulate(attribute_mean, attribute_std_dev, infection_mean, infection_std_dev, num_draws)
        
        #wtp_CI_upper = wtp_mean +  1.96 * np.std(wtp_draws) / np.sqrt(n)
        #wtp_CI_lower = wtp_mean - 1.96 * np.std(wtp_draws) / np.sqrt(n)


        # Add results to the appropriate row in the DataFrame
        results_gender.loc[index, 'WTP_mean'] = wtp_mean
        #results_gender.loc[index, 'WTP_ste'] = 1.96 * np.std(wtp_draws) / np.sqrt(n)
        results_gender.loc[index, 'WTP_CI_lower'] = wtp_CI_lower
        results_gender.loc[index, 'WTP_CI_upper'] = wtp_CI_upper


        print(f"Attribute: {index}")
        print(f"Estimated mean of WTP: {wtp_mean}")
        print(f"Estimated lower CI of WTP: {wtp_CI_lower}")
        print(f"Estimated upper CI of WTP: {wtp_CI_upper}")

Political segmentation

In [ ]:
num_draws = 100000
n  = num_participants
# Retrieve the mean and std dev of the price coefficient
infection_index_D = results_political.index.get_loc('Number_of_infections')
infection_mean_D = results_political.iloc[infection_index_D]['Value']
infection_std_dev_D = results_political.iloc[infection_index_D]['Std err']

infection_index_I = results_political.index.get_loc('Number_of_infections_Independent')
infection_mean_I = results_political.iloc[infection_index_I]['Value']
infection_std_dev_I = results_political.iloc[infection_index_I]['Std err']

infection_index_R = results_political.index.get_loc('Number_of_infections_Republican')
infection_mean_R = results_political.iloc[infection_index_R]['Value']
infection_std_dev_R = results_political.iloc[infection_index_R]['Std err']

infection_index_O = results_political.index.get_loc('Number_of_infections_Other')
infection_mean_O = results_political.iloc[infection_index_O]['Value']
infection_std_dev_O = results_political.iloc[infection_index_O]['Std err']

# Iterate over DataFrame rows and simulate WTP draws
for index, row in results_political.iterrows():
    if "Independent" in index:
            infection_mean = infection_mean_I
            infection_std_dev = infection_std_dev_I
            infection_index = infection_index_I
    elif "Republican" in index:
            infection_mean = infection_mean_R
            infection_std_dev = infection_std_dev_R
            infection_index = infection_index_R
    elif "Other" in index:
            infection_mean = infection_mean_O
            infection_std_dev = infection_std_dev_O
            infection_index = infection_index_O
    else:   
            infection_mean = infection_mean_D
            infection_std_dev = infection_std_dev_D
            infection_index = infection_index_D
    attribute_mean = row["Value"]
    attribute_std_dev = row["Std err"]   
    
    # Skip Price attribute
    if index != infection_index:
        if attribute_std_dev == -0.0:
            attribute_std_dev = 0.0
        wtp_mean, wtp_CI_lower, wtp_CI_upper = calculate_wta_and_simulate(attribute_mean, attribute_std_dev, infection_mean, infection_std_dev, num_draws)


        # Add results to the appropriate row in the DataFrame
        results_political.loc[index, 'WTP_mean'] = wtp_mean
        results_political.loc[index, 'WTP_CI_lower'] = wtp_CI_lower
        results_political.loc[index, 'WTP_CI_upper'] = wtp_CI_upper

Age

In [ ]:

num_draws = 100000
n  = num_participants
infection_index_Y = results_age.index.get_loc('Number_of_infections')
infection_mean_Y = results_age.iloc[infection_index_Y]['Value']
infection_std_dev_Y = results_age.iloc[infection_index_Y]['Std err']

infection_index_O = results_age.index.get_loc('Number_of_infections_≥ 65')
infection_mean_O = results_age.iloc[infection_index_O]['Value']
infection_std_dev_O = results_age.iloc[infection_index_O]['Std err']


# Iterate over DataFrame rows and simulate WTP draws
for index, row in results_age.iterrows():
    if "≥ 65" in index:
        infection_mean = infection_mean_O
        infection_std_dev = infection_std_dev_O
        infection_index = infection_index_O
    else:   
        infection_mean = infection_mean_Y
        infection_std_dev = infection_std_dev_Y
        infection_index = infection_index_Y
    attribute_mean = row["Value"]
    attribute_std_dev = row["Std err"]   
    
    # Skip Price attribute
    if index != infection_index:
        if attribute_std_dev == -0.0:
            attribute_std_dev = 0.0
        wtp_mean, wtp_CI_lower, wtp_CI_upper = calculate_wta_and_simulate(attribute_mean, attribute_std_dev, infection_mean, infection_std_dev, num_draws)

        # Add results to the appropriate row in the DataFrame
        results_age.loc[index, 'WTP_mean'] = wtp_mean
        #results_age.loc[index, 'WTP_ste'] = 1.96 * np.std(wtp_draws) / np.sqrt(n)
        results_age.loc[index, 'WTP_CI_lower'] = wtp_CI_lower
        results_age.loc[index, 'WTP_CI_upper'] = wtp_CI_upper


        print(f"Attribute: {index}")
        print(f"Estimated mean of WTP: {wtp_mean}")
        print(f"Estimated lower CI of WTP: {wtp_CI_lower}")
        print(f"Estimated upper CI of WTP: {wtp_CI_upper}")


Income

In [ ]:

num_draws = 100000
n  = num_participants
# Retrieve the mean and std dev of the price coefficient

infection_index_L = results_income.index.get_loc('Number_of_infections')
infection_mean_L = results_income.iloc[infection_index_L]['Value']
infection_std_dev_L = results_income.iloc[infection_index_L]['Std err']

infection_index_M = results_income.index.get_loc('Number_of_infections_$35,000 - 75,000')
infection_mean_M = results_income.iloc[infection_index_M]['Value']
infection_std_dev_M = results_income.iloc[infection_index_M]['Std err']

infection_index_H = results_income.index.get_loc('Number_of_infections_> $150,000')
infection_mean_H = results_income.iloc[infection_index_H]['Value']
infection_std_dev_H = results_income.iloc[infection_index_H]['Std err']


# Iterate over DataFrame rows and simulate WTP draws
for index, row in results_income.iterrows():
    if "$35,000 - 75,000" in index:
        infection_mean = infection_mean_M
        infection_std_dev = infection_std_dev_M
        infection_index = infection_index_M
    elif "> $150,000" in index:
        infection_mean = infection_mean_H
        infection_std_dev = infection_std_dev_H
        infection_index = infection_index_H
    else:   
        infection_mean = infection_mean_L
        infection_std_dev = infection_std_dev_L
        infection_index = infection_index_L
    attribute_mean = row["Value"]
    attribute_std_dev = row["Std err"]   
    
    # Skip Price attribute
    if index != infection_index:
        if attribute_std_dev == -0.0:
            attribute_std_dev = 0.0
        if infection_std_dev == 0.0:
            infection_std_dev = 0.0
        wtp_mean, wtp_CI_lower, wtp_CI_upper = calculate_wta_and_simulate(attribute_mean, attribute_std_dev, infection_mean, infection_std_dev, num_draws)
        
        #wtp_CI_upper = wtp_mean +  1.96 * np.std(wtp_draws) / np.sqrt(n)
        #wtp_CI_lower = wtp_mean - 1.96 * np.std(wtp_draws) / np.sqrt(n)


        # Add results to the appropriate row in the DataFrame
        results_income.loc[index, 'WTP_mean'] = wtp_mean
        #results_income.loc[index, 'WTP_ste'] = 1.96 * np.std(wtp_draws) / np.sqrt(n)
        results_income.loc[index, 'WTP_CI_lower'] = wtp_CI_lower
        results_income.loc[index, 'WTP_CI_upper'] = wtp_CI_upper


        print(f"Attribute: {index}")
        print(f"Estimated mean of WTP: {wtp_mean}")
        print(f"Estimated lower CI of WTP: {wtp_CI_lower}")
        print(f"Estimated upper CI of WTP: {wtp_CI_upper}")


Chronic

In [ ]:

num_draws = 100000
n  = num_participants
infection_index_Yes = results_chronic.index.get_loc('Number_of_infections')
infection_mean_Yes = results_chronic.iloc[infection_index_Yes]['Value']
infection_std_dev_Yes = results_chronic.iloc[infection_index_Yes]['Std err']

infection_index_No = results_chronic.index.get_loc('Number_of_infections_No')
infection_mean_No = results_chronic.iloc[infection_index_No]['Value']
infection_std_dev_No = results_chronic.iloc[infection_index_No]['Std err']

infection_index_No_Answer = results_chronic.index.get_loc('Number_of_infections_Prefer not to answer')
infection_mean_No_Answer = results_chronic.iloc[infection_index_No_Answer]['Value']
infection_std_dev_No_Answer = results_chronic.iloc[infection_index_No_Answer]['Std err']

# Iterate over DataFrame rows and simulate WTP draws
for index, row in results_chronic.iterrows():
    if "No" in index:
            infection_mean = infection_mean_No
            infection_std_dev = infection_std_dev_No
            infection_index = infection_index_No
    elif "Prefer" in index:
            infection_mean = infection_mean_No_Answer
            infection_std_dev = infection_std_dev_No_Answer
            infection_index = infection_index_No_Answer
    else:   
            infection_mean = infection_mean_Yes
            infection_std_dev = infection_std_dev_Yes
            infection_index = infection_index_Yes
    attribute_mean = row["Value"]
    attribute_std_dev = row["Std err"]   
    
    # Skip Price attribute
    if index != infection_index:
        if attribute_std_dev == -0.0:
            attribute_std_dev = 0.0
        wtp_mean, wtp_CI_lower, wtp_CI_upper = calculate_wta_and_simulate(attribute_mean, attribute_std_dev, infection_mean, infection_std_dev, num_draws)


        # Add results to the appropriate row in the DataFrame
        results_chronic.loc[index, 'WTP_mean'] = wtp_mean
        #results_chronic.loc[index, 'WTP_ste'] = 1.96 * np.std(wtp_draws) / np.sqrt(n)
        results_chronic.loc[index, 'WTP_CI_lower'] = wtp_CI_lower
        results_chronic.loc[index, 'WTP_CI_upper'] = wtp_CI_upper


Race

In [ ]:
num_draws = 100000
n  =num_participants
# Retrieve the mean and std dev of the price coefficient
infection_index = results_race.index.get_loc('Number_of_infections')
infection_mean = results_race.iloc[infection_index]['Value']
infection_std_dev = results_race.iloc[infection_index]['Std err']   

# Iterate over DataFrame rows and simulate WTP draws
for index, row in results_race.iterrows():
    attribute_mean = row["Value"]
    attribute_std_dev = row["Std err"]   
    
    # Skip Price attribute
    if index != infection_index:
        if attribute_std_dev == -0.0:
            attribute_std_dev = 0.0
        wtp_mean, wtp_CI_lower, wtp_CI_upper = calculate_wta_and_simulate(attribute_mean, attribute_std_dev, infection_mean, infection_std_dev, num_draws)


        # Add results to the appropriate row in the DataFrame
        results_race.loc[index, 'WTP_mean'] = wtp_mean
        #results_race.loc[index, 'WTP_ste'] = 1.96 * np.std(wtp_draws) / np.sqrt(n)
        results_race.loc[index, 'WTP_CI_lower'] = wtp_CI_lower
        results_race.loc[index, 'WTP_CI_upper'] = wtp_CI_upper


        print(f"Attribute: {index}")
        print(f"Estimated mean of WTP: {wtp_mean}")
        print(f"Estimated lower CI of WTP: {wtp_CI_lower}")
        print(f"Estimated upper CI of WTP: {wtp_CI_upper}")



Vaccination

In [ ]:
num_draws = 100000
n = num_participants
# Retrieve the mean and std dev of the infection coefficient
infection_index_Yes = results_vaccination.index.get_loc('Number_of_infections')
infection_mean_Yes = results_vaccination.iloc[infection_index_Yes]['Value']
infection_std_dev_Yes = results_vaccination.iloc[infection_index_Yes]['Std err']

infection_index_No = results_vaccination.index.get_loc('Number_of_infections_No')
infection_mean_No = results_vaccination.iloc[infection_index_No]['Value']
infection_std_dev_No = results_vaccination.iloc[infection_index_No]['Std err']

# Iterate over DataFrame rows and simulate WTP draws
for index, row in results_vaccination.iterrows():
    if "No" in index:
            infection_mean = infection_mean_No
            infection_std_dev = infection_std_dev_No
            infection_index = infection_index_No
    else:   
            infection_mean = infection_mean_Yes
            infection_std_dev = infection_std_dev_Yes
            infection_index = infection_index_Yes
    attribute_mean = row["Value"]
    attribute_std_dev = row["Std err"]

    # Skip Infection attribute
    if index != infection_index:
        if attribute_std_dev == -0.0:
            attribute_std_dev = 0.0
        wtp_mean, wtp_CI_lower, wtp_CI_upper = calculate_wta_and_simulate(attribute_mean, attribute_std_dev, infection_mean, infection_std_dev, num_draws)

        # Add results to the appropriate row in the DataFrame
        results_vaccination.loc[index, 'WTP_mean'] = wtp_mean
        results_vaccination.loc[index, 'WTP_CI_lower'] = wtp_CI_lower
        results_vaccination.loc[index, 'WTP_CI_upper'] = wtp_CI_upper

Child

In [ ]:
num_draws = 100000
n = len(study_data)
# Retrieve the mean and std dev of the price coefficient
infection_index_Yes = results_child.index.get_loc('Number_of_infections')
infection_mean_Yes = results_child.iloc[infection_index_Yes]['Value']
infection_std_dev_Yes = results_child.iloc[infection_index_Yes]['Std err']

infection_index_No = results_child.index.get_loc('Number_of_infections_No')
infection_mean_No = results_child.iloc[infection_index_No]['Value']
infection_std_dev_No = results_child.iloc[infection_index_No]['Std err']


# Iterate over DataFrame rows and simulate WTP draws
for index, row in results_child.iterrows():
    print(index)
    if "No" in index:
        infection_mean = infection_mean_No
        infection_std_dev = infection_std_dev_No
        infection_index = infection_index_No
    else:   
        infection_mean = infection_mean_Yes
        infection_std_dev = infection_std_dev_Yes
        infection_index = infection_index_Yes
    attribute_mean = row["Value"]
    attribute_std_dev = row["Std err"] 
    
    # Skip Price attribute
    if index != infection_index:
        if attribute_std_dev == -0.0:
            attribute_std_dev = 0.0
        wtp_mean, wtp_CI_lower, wtp_CI_upper = calculate_wta_and_simulate(attribute_mean, attribute_std_dev, infection_mean, infection_std_dev, num_draws)
        
        # Add results to the appropriate row in the DataFrame
        results_child.loc[index, 'WTP_mean'] = wtp_mean
        results_child.loc[index, 'WTP_CI_lower'] = wtp_CI_lower
        results_child.loc[index, 'WTP_CI_upper'] = wtp_CI_upper
    

Assisted Living

In [ ]:
num_draws = 100000
n = num_participants
# Retrieve the mean and std dev of the price coefficient
infection_index_Yes = results_assisted_living.index.get_loc('Number_of_infections')
infection_mean_Yes = results_assisted_living.iloc[infection_index_Yes]['Value']
infection_std_dev_Yes = results_assisted_living.iloc[infection_index_Yes]['Std err']

infection_index_No = results_assisted_living.index.get_loc('Number_of_infections_No')
infection_mean_No = results_assisted_living.iloc[infection_index_No]['Value']
infection_std_dev_No = results_assisted_living.iloc[infection_index_No]['Std err']

# Iterate over DataFrame rows and simulate WTP draws
for index, row in results_assisted_living.iterrows():
    if "No" in index:
            infection_mean = infection_mean_No
            infection_std_dev = infection_std_dev_No
            infection_index = infection_index_No
    else:   
            infection_mean = infection_mean_Yes
            infection_std_dev = infection_std_dev_Yes
            infection_index = infection_index_Yes
    attribute_mean = row["Value"]
    attribute_std_dev = row["Std err"] 
    
    # Skip Price attribute
    if index != infection_index:
        if attribute_std_dev == -0.0:
            attribute_std_dev = 0.0
        wtp_mean, wtp_CI_lower, wtp_CI_upper = calculate_wta_and_simulate(attribute_mean, attribute_std_dev, infection_mean, infection_std_dev, num_draws)
        
        # Add results to the appropriate row in the DataFrame
        results_assisted_living.loc[index, 'WTP_mean'] = wtp_mean
        results_assisted_living.loc[index, 'WTP_CI_lower'] = wtp_CI_lower
        results_assisted_living.loc[index, 'WTP_CI_upper'] = wtp_CI_upper



Vulnerable Contact 

In [ ]:
num_draws = 100000
n = num_participants
# Retrieve the mean and std dev of the price coefficient
infection_index_Yes = results_vulnerable_contact.index.get_loc('Number_of_infections')
infection_mean_Yes = results_vulnerable_contact.iloc[infection_index_Yes]['Value']
infection_std_dev_Yes = results_vulnerable_contact.iloc[infection_index_Yes]['Std err']

infection_index_No = results_vulnerable_contact.index.get_loc('Number_of_infections_No')
infection_mean_No = results_vulnerable_contact.iloc[infection_index_No]['Value']
infection_std_dev_No = results_vulnerable_contact.iloc[infection_index_No]['Std err']

# Iterate over DataFrame rows and simulate WTP draws
for index, row in results_vulnerable_contact.iterrows():
    if "No" in index:
            infection_mean = infection_mean_No
            infection_std_dev = infection_std_dev_No
            infection_index = infection_index_No
    else:   
            infection_mean = infection_mean_Yes
            infection_std_dev = infection_std_dev_Yes
            infection_index = infection_index_Yes
    attribute_mean = row["Value"]
    attribute_std_dev = row["Std err"] 
    
    # Skip Price attribute
    if index != infection_index:
        if attribute_std_dev == -0.0:
            attribute_std_dev = 0.0
        wtp_mean, wtp_CI_lower, wtp_CI_upper = calculate_wta_and_simulate(attribute_mean, attribute_std_dev, infection_mean, infection_std_dev, num_draws)
        
        # Add results to the appropriate row in the DataFrame
        results_vulnerable_contact.loc[index, 'WTP_mean'] = wtp_mean
        results_vulnerable_contact.loc[index, 'WTP_CI_lower'] = wtp_CI_lower
        results_vulnerable_contact.loc[index, 'WTP_CI_upper'] = wtp_CI_upper

Residence 

In [ ]:
num_draws = 100000
n = num_participants
# Retrieve the mean and std dev of the price coefficient
infection_index_Urban = results_residence.index.get_loc('Number_of_infections')
infection_mean_Urban = results_residence.iloc[infection_index_Urban]['Value']
infection_std_dev_Urban = results_residence.iloc[infection_index_Urban]['Std err']

infection_index_Suburban = results_residence.index.get_loc('Number_of_infections_Suburban')
infection_mean_Suburban = results_residence.iloc[infection_index_Suburban]['Value']
infection_std_dev_Suburban = results_residence.iloc[infection_index_Suburban]['Std err']

infection_index_Rural = results_residence.index.get_loc('Number_of_infections_Rural')
infection_mean_Rural = results_residence.iloc[infection_index_Rural]['Value']
infection_std_dev_Rural = results_residence.iloc[infection_index_Rural]['Std err']

# Iterate over DataFrame rows and simulate WTP draws
for index, row in results_residence.iterrows():
    if "Suburban" in index:
            infection_mean = infection_mean_Suburban
            infection_std_dev = infection_std_dev_Suburban
            infection_index = infection_index_Suburban
    elif "Rural" in index:
            infection_mean = infection_mean_Rural
            infection_std_dev = infection_std_dev_Rural
            infection_index = infection_index_Rural
    else:   
            infection_mean = infection_mean_Urban
            infection_std_dev = infection_std_dev_Urban
            infection_index = infection_index_Urban
    attribute_mean = row["Value"]
    attribute_std_dev = row["Std err"] 
    
    # Skip Price attribute
    if index != infection_index:
        if attribute_std_dev == -0.0:
            attribute_std_dev = 0.0
        wtp_mean, wtp_CI_lower, wtp_CI_upper = calculate_wta_and_simulate(attribute_mean, attribute_std_dev, infection_mean, infection_std_dev, num_draws)
        
        # Add results to the appropriate row in the DataFrame
        results_residence.loc[index, 'WTP_mean'] = wtp_mean
        results_residence.loc[index, 'WTP_CI_lower'] = wtp_CI_lower
        results_residence.loc[index, 'WTP_CI_upper'] = wtp_CI_upper


News 

In [ ]:
num_draws = 100000
n = num_participants
# Retrieve the mean and std dev of the price coefficient
infection_index_Social = results_news.index.get_loc('Number_of_infections')
infection_mean_Social = results_news.iloc[infection_index_Social]['Value']
infection_std_dev_Social = results_news.iloc[infection_index_Social]['Std err']


infection_index_TV = results_news.index.get_loc('Number_of_infections_TV (including cable)')
infection_mean_TV = results_news.iloc[infection_index_TV]['Value']
infection_std_dev_TV = results_news.iloc[infection_index_TV]['Std err']

infection_index_Radio = results_news.index.get_loc('Number_of_infections_Radio or podcasts')
infection_mean_Radio = results_news.iloc[infection_index_Radio]['Value']
infection_std_dev_Radio = results_news.iloc[infection_index_Radio]['Std err']


infection_index_Apps = results_news.index.get_loc('Number_of_infections_News apps or websites')
infection_mean_Apps = results_news.iloc[infection_index_Apps]['Value']
infection_std_dev_Apps = results_news.iloc[infection_index_Apps]['Std err']

infection_index_Other = results_news.index.get_loc('Number_of_infections_Other')
infection_mean_Other = results_news.iloc[infection_index_Other]['Value']
infection_std_dev_Other = results_news.iloc[infection_index_Other]['Std err']



# Iterate over DataFrame rows and simulate WTP draws
for index, row in results_news.iterrows():
        if "Print media (newspapers, journals)" in index:
            infection_mean = infection_mean_Print
            infection_std_dev = infection_std_dev_Print
            infection_index = infection_index_Print
        elif "TV" in index:
            infection_mean = infection_mean_TV
            infection_std_dev = infection_std_dev_TV
            infection_index = infection_index_TV
        elif "Radio" in index:
            infection_mean = infection_mean_Radio
            infection_std_dev = infection_std_dev_Radio
            infection_index = infection_index_Radio
        elif "Do not" in index:
            infection_mean = infection_mean_No
            infection_std_dev = infection_std_dev_No
            infection_index = infection_index_No
        elif "News apps" in index:
            infection_mean = infection_mean_Apps
            infection_std_dev = infection_std_dev_Apps
            infection_index = infection_index_Apps
        elif "Other" in index:
            infection_mean = infection_mean_Other
            infection_std_dev = infection_std_dev_Other
            infection_index = infection_index_Other
        else:   
            infection_mean = infection_mean_Social
            infection_std_dev = infection_std_dev_Social
            infection_index = infection_index_Social
        attribute_mean = row["Value"]
        attribute_std_dev = row["Std err"] 
    
    # Skip Price attribute
        if index != infection_index:
                if attribute_std_dev == -0.0:
                        attribute_std_dev = 0.0
                wtp_mean, wtp_CI_lower, wtp_CI_upper = calculate_wta_and_simulate(attribute_mean, attribute_std_dev, infection_mean, infection_std_dev, num_draws)
                
                # Add results to the appropriate row in the DataFrame
                results_news.loc[index, 'WTP_mean'] = wtp_mean
                results_news.loc[index, 'WTP_CI_lower'] = wtp_CI_lower
                results_news.loc[index, 'WTP_CI_upper'] = wtp_CI_upper

                print(f"Attribute: {index}")
                print(f"Estimated mean of WTP: {wtp_mean}")
                print(f"Estimated lower CI of WTP: {wtp_CI_lower}")
                print(f"Estimated upper CI of WTP: {wtp_CI_upper}")

Self employed

In [ ]:
num_draws = 100000
n = num_participants
# Retrieve the mean and std dev of the price coefficient
infection_index_Yes = results_self_employed.index.get_loc('Number_of_infections')
infection_mean_Yes = results_self_employed.iloc[infection_index_Yes]['Value']
infection_std_dev_Yes = results_self_employed.iloc[infection_index_Yes]['Std err']

infection_index_No = results_self_employed.index.get_loc('Number_of_infections_No')
infection_mean_No = results_self_employed.iloc[infection_index_No]['Value']
infection_std_dev_No = results_self_employed.iloc[infection_index_No]['Std err']


# Iterate over DataFrame rows and simulate WTP draws
for index, row in results_self_employed.iterrows():
    if "No" in index:
            infection_mean = infection_mean_No
            infection_std_dev = infection_std_dev_No
            infection_index = infection_index_No

    else:   
            infection_mean = infection_mean_Yes
            infection_std_dev = infection_std_dev_Yes
            infection_index = infection_index_Yes
    attribute_mean = row["Value"]
    attribute_std_dev = row["Std err"] 
    
    # Skip Price attribute
    if index != infection_index:
        if attribute_std_dev == -0.0:
            attribute_std_dev = 0.0
        wtp_mean, wtp_CI_lower, wtp_CI_upper = calculate_wta_and_simulate(attribute_mean, attribute_std_dev, infection_mean, infection_std_dev, num_draws)
        
        # Add results to the appropriate row in the DataFrame
        results_self_employed.loc[index, 'WTP_mean'] = wtp_mean
        results_self_employed.loc[index, 'WTP_CI_lower'] = wtp_CI_lower
        results_self_employed.loc[index, 'WTP_CI_upper'] = wtp_CI_upper

        print(f"Attribute: {index}")
        print(f"Estimated mean of WTP: {wtp_mean}")
        print(f"Estimated lower CI of WTP: {wtp_CI_lower}")
        print(f"Estimated upper CI of WTP: {wtp_CI_upper}")


Remote

In [ ]:
num_draws = 100000
n = num_participants
# Retrieve the mean and std dev of the price coefficient
infection_index_Yes = results_remote.index.get_loc('Number_of_infections')
infection_mean_Yes = results_remote.iloc[infection_index_Yes]['Value']
infection_std_dev_Yes = results_remote.iloc[infection_index_Yes]['Std err']

infection_index_No = results_remote.index.get_loc('Number_of_infections_No')
infection_mean_No = results_remote.iloc[infection_index_No]['Value']
infection_std_dev_No = results_remote.iloc[infection_index_No]['Std err']

# Iterate over DataFrame rows and simulate WTP draws
for index, row in results_remote.iterrows():
    if "No" in index:
            infection_mean = infection_mean_No
            infection_std_dev = infection_std_dev_No
            infection_index = infection_index_No

    else:   
            infection_mean = infection_mean_Yes
            infection_std_dev = infection_std_dev_Yes
            infection_index = infection_index_Yes
    attribute_mean = row["Value"]
    attribute_std_dev = row["Std err"] 
    
    # Skip Price attribute
    if index != infection_index:
        if attribute_std_dev == -0.0:
            attribute_std_dev = 0.0
        wtp_mean, wtp_CI_lower, wtp_CI_upper = calculate_wta_and_simulate(attribute_mean, attribute_std_dev, infection_mean, infection_std_dev, num_draws)
        
        # Add results to the appropriate row in the DataFrame
        results_remote.loc[index, 'WTP_mean'] = wtp_mean
        results_remote.loc[index, 'WTP_CI_lower'] = wtp_CI_lower
        results_remote.loc[index, 'WTP_CI_upper'] = wtp_CI_upper

        print(f"Attribute: {index}")
        print(f"Estimated mean of WTP: {wtp_mean}")
        print(f"Estimated lower CI of WTP: {wtp_CI_lower}")
        print(f"Estimated upper CI of WTP: {wtp_CI_upper}")


Vehicle

In [ ]:
num_draws = 100000
n = num_participants
# Retrieve the mean and std dev of the price coefficient
infection_index_Yes = results_vehicle.index.get_loc('Number_of_infections')
infection_mean_Yes = results_vehicle.iloc[infection_index_Yes]['Value']
infection_std_dev_Yes = results_vehicle.iloc[infection_index_Yes]['Std err']

infection_index_No = results_vehicle.index.get_loc('Number_of_infections_No')
infection_mean_No = results_vehicle.iloc[infection_index_No]['Value']
infection_std_dev_No = results_vehicle.iloc[infection_index_No]['Std err']

# Iterate over DataFrame rows and simulate WTP draws
for index, row in results_vehicle.iterrows():
    if "No" in index:
            infection_mean = infection_mean_No
            infection_std_dev = infection_std_dev_No
            infection_index = infection_index_No

    else:   
            infection_mean = infection_mean_Yes
            infection_std_dev = infection_std_dev_Yes
            infection_index = infection_index_Yes
    attribute_mean = row["Value"]
    attribute_std_dev = row["Std err"] 
    
    # Skip Price attribute
    if index != infection_index:
        if attribute_std_dev == -0.0:
            attribute_std_dev = 0.0
        wtp_mean, wtp_CI_lower, wtp_CI_upper = calculate_wta_and_simulate(attribute_mean, attribute_std_dev, infection_mean, infection_std_dev, num_draws)
        
        # Add results to the appropriate row in the DataFrame
        results_vehicle.loc[index, 'WTP_mean'] = wtp_mean
        results_vehicle.loc[index, 'WTP_CI_lower'] = wtp_CI_lower
        results_vehicle.loc[index, 'WTP_CI_upper'] = wtp_CI_upper

        print(f"Attribute: {index}")
        print(f"Estimated mean of WTP: {wtp_mean}")
        print(f"Estimated lower CI of WTP: {wtp_CI_lower}")
        print(f"Estimated upper CI of WTP: {wtp_CI_upper}")

Pregnant

In [ ]:
num_draws = 100000
n = num_participants
# Retrieve the mean and std dev of the price coefficient
infection_index_Yes = results_pregnant.index.get_loc('Number_of_infections')
infection_mean_Yes = results_pregnant.iloc[infection_index_Yes]['Value']
infection_std_dev_Yes = results_pregnant.iloc[infection_index_Yes]['Std err']

infection_index_No = results_pregnant.index.get_loc('Number_of_infections_No')
infection_mean_No = results_pregnant.iloc[infection_index_No]['Value']
infection_std_dev_No = results_pregnant.iloc[infection_index_No]['Std err']


# Iterate over DataFrame rows and simulate WTP draws
for index, row in results_pregnant.iterrows():
    if "No" in index:
            infection_mean = infection_mean_No
            infection_std_dev = infection_std_dev_No
            infection_index = infection_index_No

    else:   
            infection_mean = infection_mean_Yes
            infection_std_dev = infection_std_dev_Yes
            infection_index = infection_index_Yes
    attribute_mean = row["Value"]
    attribute_std_dev = row["Std err"] 
    
    # Skip Price attribute
    if index != infection_index:
        if attribute_std_dev == -0.0:
            attribute_std_dev = 0.0
        wtp_mean, wtp_CI_lower, wtp_CI_upper = calculate_wta_and_simulate(attribute_mean, attribute_std_dev, infection_mean, infection_std_dev, num_draws)
        
        # Add results to the appropriate row in the DataFrame
        results_pregnant.loc[index, 'WTP_mean'] = wtp_mean
        results_pregnant.loc[index, 'WTP_CI_lower'] = wtp_CI_lower
        results_pregnant.loc[index, 'WTP_CI_upper'] = wtp_CI_upper

        print(f"Attribute: {index}")
        print(f"Estimated mean of WTP: {wtp_mean}")
        print(f"Estimated lower CI of WTP: {wtp_CI_lower}")
        print(f"Estimated upper CI of WTP: {wtp_CI_upper}")

Education

In [ ]:
num_draws = 100000
n = num_participants
# Retrieve the mean and std dev of the price coefficient
infection_index_No_college = results_education.index.get_loc('Number_of_infections')
infection_mean_No_college = results_education.iloc[infection_index_No_college]['Value']
infection_std_dev_No_college = results_education.iloc[infection_index_No_college]['Std err']

infection_index_Grad = results_education.index.get_loc('Number_of_infections_Postgraduate degree')
infection_mean_Grad = results_education.iloc[infection_index_Grad]['Value']
infection_std_dev_Grad = results_education.iloc[infection_index_Grad]['Std err']


# Iterate over DataFrame rows and simulate WTP draws
for index, row in results_education.iterrows():
    if "Grad" in index:
            infection_mean = infection_mean_Grad
            infection_std_dev = infection_std_Grad
            infection_index = infection_index_Grad
    else:   
            infection_mean = infection_mean_No_college
            infection_std_dev = infection_std_dev_No_college
            infection_index = infection_index_No_college
    attribute_mean = row["Value"]
    attribute_std_dev = row["Std err"] 
    
    # Skip Price attribute
    if index != infection_index:
        if attribute_std_dev == -0.0:
            attribute_std_dev = 0.0
        wtp_mean, wtp_CI_lower, wtp_CI_upper = calculate_wta_and_simulate(attribute_mean, attribute_std_dev, infection_mean, infection_std_dev, num_draws)
        
        # Add results to the appropriate row in the DataFrame
        results_education.loc[index, 'WTP_mean'] = wtp_mean
        results_education.loc[index, 'WTP_CI_lower'] = wtp_CI_lower
        results_education.loc[index, 'WTP_CI_upper'] = wtp_CI_upper

        print(f"Attribute: {index}")
        print(f"Estimated mean of WTP: {wtp_mean}")
        print(f"Estimated lower CI of WTP: {wtp_CI_lower}")
        print(f"Estimated upper CI of WTP: {wtp_CI_upper}")

Health insurance

In [ ]:
num_draws = 100000
n = num_participants
# Retrieve the mean and std dev of the price coefficient
infection_index_Yes = results_health_insurance.index.get_loc('Number_of_infections')
infection_mean_Yes = results_health_insurance.iloc[infection_index_Yes]['Value']
infection_std_dev_Yes = results_health_insurance.iloc[infection_index_Yes]['Std err']

infection_index_No = results_health_insurance.index.get_loc('Number_of_infections_No')
infection_mean_No = results_health_insurance.iloc[infection_index_No]['Value']
infection_std_dev_No = results_health_insurance.iloc[infection_index_No]['Std err']

# Iterate over DataFrame rows and simulate WTP draws
for index, row in results_health_insurance.iterrows():
    if "No" in index:
            infection_mean = infection_mean_No
            infection_std_dev = infection_std_dev_No
            infection_index = infection_index_No
    else:   
            infection_mean = infection_mean_Yes
            infection_std_dev = infection_std_dev_Yes
            infection_index = infection_index_Yes
    attribute_mean = row["Value"]
    attribute_std_dev = row["Std err"] 
    
    # Skip Price attribute
    if index != infection_index:
        if attribute_std_dev == -0.0:
            attribute_std_dev = 0.0
        wtp_mean, wtp_CI_lower, wtp_CI_upper = calculate_wta_and_simulate(attribute_mean, attribute_std_dev, infection_mean, infection_std_dev, num_draws)
        
        # Add results to the appropriate row in the DataFrame
        results_health_insurance.loc[index, 'WTP_mean'] = wtp_mean
        results_health_insurance.loc[index, 'WTP_CI_lower'] = wtp_CI_lower
        results_health_insurance.loc[index, 'WTP_CI_upper'] = wtp_CI_upper

        print(f"Attribute: {index}")
        print(f"Estimated mean of WTP: {wtp_mean}")
        print(f"Estimated lower CI of WTP: {wtp_CI_lower}")
        print(f"Estimated upper CI of WTP: {wtp_CI_upper}")

# Compare differences between surveys

In [ ]:
results_no_vaccine = pd.read_csv('results_drop_first_level_no_vaccine.csv', index_col= 0)
results_vaccine = pd.read_csv('results_drop_first_level_vaccine.csv', index_col= 0)

In [ ]:

def compare_coefficients_between_dfs(df1, df2):
    # Extract coefficients and standard errors
    coefficients1 = df1['Value'].values
    std_errors1 = df1['Rob. Std err'].values
    
    coefficients2 = df2['Value'].values
    std_errors2 = df2['Rob. Std err'].values
    
    # Initialize list to store pairwise results
    comparison_results = []
    
    # Compare equivalent rows
    for i in range(len(coefficients1)):
        # Calculate t-statistic
        t_stat = (coefficients1[i] - coefficients2[i]) / np.sqrt(std_errors1[i]**2 + std_errors2[i]**2)
        
        # Degrees of freedom approximation (assuming large sample size)
        df = len(coefficients1) - 2
        
        # Calculate p-value
        p_value = 2 * (1 - t.cdf(np.abs(t_stat), df))
        
        # Store the results
        comparison_results.append({
            'Predictor': df1.index[i],  # Assuming index corresponds to predictor names
            'tStatistic': t_stat,
            'pValue': p_value
        })
    
    # Convert to DataFrame
    comparison_p_values_df = pd.DataFrame(comparison_results)
    
    # Apply Bonferroni adjustment
    num_tests = len(comparison_p_values_df)
    bonferroni_threshold = 0.05 / num_tests
    comparison_p_values_df['Bonferroni pValue'] = comparison_p_values_df['pValue'] * num_tests
    comparison_p_values_df['Bonferroni pValue'] = comparison_p_values_df['Bonferroni pValue'].clip(upper=1)
    comparison_p_values_df['Bonferroni Significant'] = comparison_p_values_df['Bonferroni pValue'] < bonferroni_threshold
    
    # Subset to show only significant comparisons
    significant_comparisons_df = comparison_p_values_df[comparison_p_values_df['Bonferroni Significant']]
    
    return significant_comparisons_df

# Perform the comparison between the two dataframes
significant_comparisons = compare_coefficients_between_dfs(results_no_vaccine, results_vaccine)

# Display the significant results
print("Significant Differences Between Equivalent Rows (No Vaccine vs. Vaccine):")
print(significant_comparisons)